# Машинное обучение, ФКН ВШЭ

## Практическое задание 4. Классификация

### Общая информация
Дата выдачи: 16.11.2024

Мягкий дедлайн: 28.11.2024

Жесткий дедлайн: 02.12.2024

### О задании

В этом задании вы:
- ознакомитесь с тем, что происходит "внутри" метода опорных векторов и логистической регрессии
- познакомитесь с калибровкой вероятности
- изучите методы трансформации переменных и методы отбора признаков
- попробуете оценить экономический эффект модели

----

#### Самостоятельная оценка результатов

Для удобства проверки, исходя из набора решенных задач, посчитайте свою максимальную оценку.

**Оценка**:

### Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

# Часть 1. SVM, LR и калибровка вероятностей (2 балла + 0.5 бонус)

In [ ]:
import numpy as np
import pandas as pd
# import polars as pl
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
# pl.Config().set_tbl_rows(100)
# pl.Config().set_tbl_cols(100)

from tqdm import tqdm

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
!pip install matplotlib_venn

#### __Задание 1.1  Сравнение методов__ (0.5 балла)



Сгенерируем синтетические данные.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# фиксируем random_state для воспроизводимости результатов
X, y = make_classification(
    n_samples=10000, n_features=10, n_informative=5, n_redundant=5, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

__Случайный классификатор__

Для начала зададим самую простую модель, которая на каждом объекте выдаёт случайный ответ. По тестовой выборке вычислим AUC-ROC, AUC-PR.

In [ ]:
from sklearn.dummy import DummyClassifier
random_classifier = DummyClassifier(strategy='uniform', random_state=42).fit(X_train, y_train)
y_random = random_classifier.predict_proba(X_test)[:,1]
y_random

**Вопрос:** решаем задачу бинарной классификации, но y\_random содержит какие-то дробные числа, а не 0/1. Почему?



**Ответ**: Потому что мы используем метод predict_proba, который вычмсляет вероятность принадлежности объекта к положительному классу

*Ниже приведен **пример** работы* со встроенными функциями `sklearn` для отрисовки ROC и PR кривых, сохранения метрик. Пайплайн можно изменять как вам удобно.

In [ ]:
from sklearn.metrics import average_precision_score

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay

In [ ]:
def depict_pr_roc(y_true, y_pred, classifier_name='Some Classifier', ax=None):
    if ax is None:
        fig, ax = plt.subplots(1, 2, figsize=(11, 5))

    print(classifier_name, 'metrics')
    PrecisionRecallDisplay.from_predictions(y_true, y_pred, ax=ax[0], name=classifier_name)
    print('AUC-PR: %.4f' % average_precision_score(y_true, y_pred))
    ax[0].set_title("PRC")
    ax[0].set_ylim(0, 1.1)

    RocCurveDisplay.from_predictions(y_true, y_pred, ax=ax[1], name=classifier_name)
    print('AUC-ROC: %.4f' % roc_auc_score(y_true, y_pred))
    ax[1].set_title("ROC")
    ax[1].set_ylim(0, 1.1)

    plt.tight_layout()
    plt.legend()
    plt.show()  

depict_pr_roc(y_test, y_random, 'Random Classifier')

In [ ]:
# dataframe для сравнения
# методов классификации по метрикам
df_metrics = pd.DataFrame(
    columns=['auc_pr', 'roc_auc_score', 'reg_const']
)
precision, recall, _ = precision_recall_curve(y_test, y_random)
# добавление очередной строки с характеристиками метода
df_metrics.loc['Random Classifier'] = [
      average_precision_score(y_test, y_random),
      roc_auc_score(y_test, y_random),
      0,
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_metrics

__Support Vector Machine (Linear Kernel)__

Обучите метод опорных векторов.

Подберите параметр регуляризации `C` с точки зрения AUC-PR (можете воспользоваться кросс-валидацией или отделить валидационную выборку от обучающей).


**Я решил подбирать с помощью кросс-валидации**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
}

In [ ]:
svc = SVC(probability=True)
grid_search = GridSearchCV(svc, param_grid, cv=5, verbose=2, scoring='average_precision') # 5 фолдов, average_precision - аналог pr-auc
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

  На тестовой части:
  - постройте ROC и PR кривые,
  - посчитайте AUC-ROC, AUC-PR

In [ ]:
y_test_pred_svc = grid_search.predict_proba(X_test)[:,1]

In [ ]:
depict_pr_roc(y_test, y_test_pred_svc, 'SVC (C=1000)')

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc
precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_svc)
pr_auc = auc(recall_vals, precision_vals)

In [ ]:
print(f'На тестовой выборке AUC-ROC = {round(roc_auc_score(y_test, y_test_pred_svc), 5)}')
print(f'На тестовой выборке AUC-PR = {round(pr_auc, 5)}')

In [ ]:
df_metrics.loc['SVC'] = [
      round(pr_auc, 5),
      round(roc_auc_score(y_test, y_test_pred_svc), 5),
      1000,
]

df_metrics

Проанализируйте, как себя ведут обе кривые:
- Что происходит при увеличении порога? Как бы вы это проинтерпретировали?
- Монотонные ли кривые? Как вы это объясните?

#### **При увеличении порога**:

**ROC кривая**: двигаясь справа налево, мы увеличиваем порог => более строго относим объекты к положительному классу => FPR падает, так как мы наращиваем свою увереннность в объектах, которые называем положительными.\
При этом мы упускаем много реально положительных объектов, которые имели веорятность ниже заданного нами высокого порога. 

**PR кривая**: при увеличении порога мы двигаемся справа налево. Recall монотонно уменьшается, так как FN растет (мы относим многие объекты с большой уверенностью модели в том, что они положительного класса, к отрицательному классу). Precision растет, но не монотонно.

#### **Монотонность:**
При уменьшении порога классификации, TPR и FPR могут только увеличиваться или оставаться неизменными => ROC кривая монотонно неубывающая.







В целом ROC-AUC демонстрирует качество разделения обоих классов, а PR-AUC качестов предсказания положительного класса


Сравните AUC-ROC и AUC-PR для вашей модели с этими же метриками для случайного классификатора.

Наша модель намного лучше случайного классификатора, так как у нее и roc-auc и auc-pr на тестовой выборке близки к 1

__Logistic Regression__


Аналогичное задание для логистической регрессии с L2 регуляризатором:


*   подберите гиперпараметр C, используя метрику AUC-PR
*   нарисуйте ROC, PR кривые для тестовой части
*   выведите метрики для тестовых данных и сравните их с результатами случайного классификатора



In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogReg = LogisticRegression(penalty='l2')
grid_search_lr = GridSearchCV(LogReg, param_grid, cv=5, verbose=2, scoring='average_precision') # 5 фолдов, average_precision - аналог pr-auc
grid_search_lr.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search_lr.best_params_)
print("Best Cross-Validation Score:", grid_search_lr.best_score_)

In [ ]:
y_test_pred_logreg = grid_search_lr.predict_proba(X_test)[:,1]

depict_pr_roc(y_test, y_test_pred_logreg, 'SVC (C=0.001)')

In [ ]:
precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr = auc(recall_vals, precision_vals)


print(f'На тестовой выборке AUC-ROC = {round(roc_auc_score(y_test, y_test_pred_logreg), 5)}')
print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr, 5)}')

In [ ]:
df_metrics.loc['Logistic Regression'] = [
      round(pr_auc_lr, 5),
      round(roc_auc_score(y_test, y_test_pred_logreg), 5),
      0.001,
]



display(df_metrics.style
    .background_gradient(subset=['auc_pr', 'roc_auc_score'], cmap='RdYlGn'))

На тестовых данных SVC заметно опережает Logreg => **в данных скорее всего есть сложные, нелинейные зависимости, которые лог регрессии сложнее распознать**

Нарисуйте ROC, PR кривые для тестовой части для всех 3 классификаторов на одном графике

**В создании этого графика мне помог deepseek**

In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc, precision_recall_curve

In [ ]:

fig = go.Figure()

fpr1, tpr1, _ = roc_curve(y_test, y_random)
auc1 = auc(fpr1, tpr1)
fig.add_trace(go.Scatter(x=fpr1, y=tpr1, 
                         name=f'Random Classifier (AUC = {auc1:.3f})',
                         line=dict(color='#FF5252', width=3)))


fpr2, tpr2, _ = roc_curve(y_test, y_test_pred_svc)
auc2 = auc(fpr2, tpr2)
fig.add_trace(go.Scatter(x=fpr2, y=tpr2,
                         name=f'SVC (AUC = {auc2:.3f})',
                         line=dict(color='#2196F3', width=3, dash='dash')))


fpr3, tpr3, _ = roc_curve(y_test, y_test_pred_logreg)
auc3 = auc(fpr3, tpr3)
fig.add_trace(go.Scatter(x=fpr3, y=tpr3,
                         name=f'Logistic Regression (AUC = {auc3:.3f})',
                         line=dict(color='#6A0DAD', width=3)))

fig.update_layout(
    title='<b>ROC кривые разных классификаторов</b>',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=1200,
    height=800
)

fig.show()

In [ ]:
# график можно посмотреть в ROC_curves_different_classifiers.png

In [ ]:
fig = go.Figure()

pr1, rec1, _ = precision_recall_curve(y_test, y_random)
as1 = average_precision_score(y_test, y_random)
fig.add_trace(go.Scatter(x=rec1, y=pr1, 
                         name=f'Random Classifier (AUC-PR = {as1:.3f})',
                         line=dict(color='#FF5252', width=3)))


pr2, rec2, _ = precision_recall_curve(y_test, y_test_pred_svc)
as2 = average_precision_score(y_test, y_test_pred_svc)
fig.add_trace(go.Scatter(x=rec2, y=pr2,
                         name=f'SVC (AUC-PR = {as2:.3f})',
                         line=dict(color='#2196F3', width=3, dash='dash')))


pr3, rec3, _ = precision_recall_curve(y_test, y_test_pred_logreg)
as3 = average_precision_score(y_test, y_test_pred_logreg)
fig.add_trace(go.Scatter(x=rec3, y=pr3,
                         name=f'Logistic Regression (AUC-PR = {as3:.3f})',
                         line=dict(color='#6A0DAD', width=3)))

fig.update_layout(
    title='<b>PR кривые разных классификаторов</b>',
    xaxis_title='Recall',
    yaxis_title='Precision',
    width=1200,
    height=800
)

fig.show()

In [ ]:
# график можно посмотреть в PR_curves_different_classifiers.png

**Вопрос:** Сравните результаты LR и SVM с точки зрения всех вычисленных критериев качества, объясните различия (если они есть).



In [ ]:
df_metrics

In [ ]:
round(0.983990 - 0.833600, 4), round(0.98652 - 0.84533, 4)

In [ ]:
y_train.sum()/len(y_train), y_test.sum()/len(y_test)

**Ответ:** в общем разделении классов у SVM большое преимущество (AUC-ROC больше на 0.14).\
Разница в PR-AUC больше, значит преимущество SVM в отделении положительного класса больше.\
В целом здесь у SVC почти идеальное качество, у LR просто хорошее.\
Если выбирать из них, то почти во всех случаях для подобной задачи SVM лучше. LR лучше подходит только если важна интерпретируемость или ограничены вычислительные ресурсы (LR обучается быстрее).


#### __Задание 1.2. Визуализация в подходах SVM, LR__ (0.5 балла)



В названии метода опорных векторов присутствуют некоторые "опорные векторы". По сути, это объекты из обучающей выборки, которые задали положение разделяющей гиперплоскости.

* Сгенерируйте синтетические данные с помощью `make_classification` __с 2 признаками__, обучите на нём метод опорных векторов. Не забудьте зафиксировать seed для воспроизводимости

* Визуализируйте разделяющую прямую, все объекты и выделите опорные векторы. Ниже есть шаблоны, можете воспользоваться ими, либо написать своё

In [ ]:
X, y = make_classification(
    n_samples=10000,          
    n_features=2,            
    n_informative=2,         #Оба признака информативные
    n_redundant=0,           
    random_state=42
)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Решил не делить, так как все равно не нужна тестовая выборка

svm_model = SVC(kernel='linear', C=1000.0, random_state=42)
svm_model.fit(X, y)

# Получаем опорные векторы
support_vectors = svm_model.support_vectors_
support_vector_indices = svm_model.support_

print(f"Количество опорных векторов: {len(support_vectors)}")
print(f"Индексы опорных векторов: {support_vector_indices}")

In [ ]:
def plot_svm_2D(X, y, model,  plot_support=True):

    # создал сетку
    xx = np.linspace(X[:,0].min(), X[:,0].max(), 30)
    yy = np.linspace(X[:,1].min(), X[:,1].max(), 30)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T

    # Ответы модели для сетки для отрисовки разделяющей прямой
    Z = model.decision_function(xy).reshape(XX.shape)

    plt.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])

    # Отрисовал выборку
    plt.scatter(
        X[y == 0, 0], X[y == 0, 1],  # Класс 0
        c='blue', label='Class 0', alpha=0.7
    )
    plt.scatter(
        X[y == 1, 0], X[y == 1, 1],  # Класс 1  
        c='red', label='Class 1', alpha=0.7
    )

    # Отрисовал опорные векторы
    if plot_support:
        plt.scatter(
            model.support_vectors_[:, 0],  # X координаты опорных векторов
            model.support_vectors_[:, 1],  # Y координаты опорных векторов
            label='support vectors',
            s=100,
            linewidth=1,
            edgecolor="blue",
            facecolors='none'
        )

    plt.legend()
    plt.show()
plot_svm_2D(X, y, svm_model)

**Вопрос:** какие объекты выделяются как "опорные"?



**Ответ:** опорными становятся объекты, лежащие на границе зазора или внутри.\
Это наиболее информативные объекты, которые находятся ближе всего к разделяющей границе и определяют положение разделяющей гиперплоскости.


В отличие от метода опорных векторов, логистическая регрессия не пытается построить разделяющую гиперплоскость с максимальным отступом, а приближает в каждой точке пространства объектов вероятность положительных ответов $p(y=+1|x)$. Попробуйте нарисовать это распределение на плоскости, не забудьте отметить на ней все объекты.

In [ ]:
logreg_model = LogisticRegression(penalty='l2', C=0.001, verbose=2, random_state=42)
logreg_model.fit(X, y)

In [ ]:
def plot_logreg_2D(X, y, model):

    # создали сетку
    xx = np.linspace(X[:,0].min(), X[:,0].max(), 100)
    yy = np.linspace(X[:,1].min(), X[:,1].max(), 100)
    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T

    # Ответы модели для сетки для отрисовки распределения
    Z = model.predict_proba(xy)[:, 1]
    Z = Z.reshape((xx.shape[0], -1)).T

    image = plt.imshow(
        Z,
        interpolation='nearest',
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        aspect='auto',
        origin='lower',
        cmap=plt.cm.PuOr_r
    )

    plt.scatter(
        X[y == 0, 0], X[y == 0, 1],  # Класс 0
        c='blue', label='Class 0', alpha=0.8, s=50, edgecolors='white'
    )
    plt.scatter(
        X[y == 1, 0], X[y == 1, 1],  # Класс 1
        c='red', label='Class 1', alpha=0.8, s=50, edgecolors='white'
    )

    # Добавляю линию уровня вероятности 0.5 (граница решения)
    contour = plt.contour(XX, YY, Z, levels=[0.5], colors='black', linewidths=2)
    plt.clabel(contour, inline=True, fontsize=12, fmt='p=0.5')

    # Добавляю дополнительные уровни вероятности для наглядности
    plt.contour(XX, YY, Z, levels=[0.25, 0.75], colors='gray', linewidths=1, linestyles='--')
    
    plt.colorbar(image, label='Вероятность класса 1: p(y=+1|x)')
    plt.xlabel('Признак 1')
    plt.ylabel('Признак 2')
    plt.title('Логистическая регрессия: распределение вероятностей')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.show()

plot_logreg_2D(X, y, logreg_model)

**Вопрос:** Как на картинке визуализирована область, где модель не уверена ($p(y=+1|x) = 0.5$)? Как это обосновать теоритечески?



**Ответ:** Область с вероятностью положительного класса = 0.5 визуализирована на картинке как белая линия.\
Сигмоида имеет наибольшую производную при P(1) = 0.5 => в этой области при небольшом изменении z вероятность принадлежности к положительному классу сильно меняется.\
Когда P(1) = 0.5, энтропия наибольшая => максимальная неопределеннось


#### __Задание 2. Калибровка вероятностей__ (1 балл)



Перейдём к оценке качества выдаваемых алгоритмами вероятностей. Начнём с калибровочных кривых.

Допустим, алгоритм возвращает некоторые числа от нуля до единицы. Хорошо ли они оценивают вероятность?

Хорошо откалиброванный  классификатор должен выдавать значения так, чтобы среди образцов, для которых он дал значение, близкое к $\alpha$, примерно $\alpha * 100 \%$ фактически принадлежали к положительному классу. (Например, если классификатор выдает 0.3 для некоторых, то 30% из них должны принадлежать классу 1)

Для построения калибровочной криовой используем следующий алгоритм:

Разобьем отрезок $[0, 1]$ на несколько маленьких отрезков одинаковой длины.

Рассмотрим $i$-й отрезок с границами $[a_i, b_i]$ и предсказания $p_1, p_2, \dots, p_k$, которые попали в него. Пусть им соответствуют истинные ответы $y_1, y_2, \dots, y_k$. Если алгоритм выдает корректные вероятности, то среди этих истинных ответов должно быть примерно $(a_i + b_i) / 2$ единиц. Иными словами, если нарисовать кривую, у которой по оси X отложены центры отрезков, а по оси Y — доли единичных ответов этих в отрезках, то она должна оказаться диагональной.

Ниже приведена функция, которая должна рисовать такие кривые. В ней допущено две ошибки — найдите и исправьте их.

In [ ]:
#def plot_calibration_curve(y_test, preds):
 #   bin_middle_points = []
  #  bin_real_ratios = []
   # n_bins = 10
    #for i in range(n_bins):
     #   l = 1.0 / n_bins * i
      #  r = 1.0 / n_bins * (i + 1)
       # bin_middle_points.append((l - r) / 2)
       # bin_real_ratios.append(np.min(y_test[(preds >= l) & (preds < r)] == 1))
    #plt.figure(figsize=(6,6))
    #plt.plot(bin_middle_points, bin_real_ratios)
    #plt.ylim([-0.05, 1.05])
    #plt.grid()

In [ ]:
def plot_calibration_curve(y_test, preds, method='SVC', color1 = 'purple'):
    bin_middle_points = []
    bin_real_ratios = []
    n_bins = 10
    y_test = np.array(y_test).flatten()
    preds = np.array(preds).flatten()
    
    for i in range(n_bins):
        l = i / n_bins
        r = (i + 1) / n_bins
        bin_middle_points.append((l + r) / 2)
        

        if i == n_bins - 1:
            mask = (preds >= l) & (preds <= r)
        else:
            mask = (preds >= l) & (preds < r)
        

        mask = np.array(mask).flatten()
        
        if mask.ndim > 1:
            mask = mask.flatten()
            
        n_in_bin = np.sum(mask)
        
        if n_in_bin > 0:
 
            y_in_bin = y_test[mask]
            ratio = np.mean(y_in_bin == 1)
            bin_real_ratios.append(ratio)
        else:
            bin_real_ratios.append(0.0)

    plt.figure(figsize=(12,8))
    plt.plot(bin_middle_points, bin_real_ratios, color=color1)
    plt.plot([0, 1], [0, 1], '--', color='gray') # идеальная калибровка
    plt.ylim([-0.05, 1.05])
    plt.grid()
    plt.xlabel('Предсказанная вероятность принадлежности к положительному классу',fontsize =12)
    plt.ylabel('Реальная вероятность принадлежности к положительному классу', fontsize =12)
    plt.title(f'Калибровочная кривая для {method}', fontsize =14, pad=15)
    plt.show()

Сгенерируйте синтетические данные аналогично использованным в самом первом задании. Постройте калибровочные кривые на тестовой части для логистической регрессии и метода опорных векторов (не забудьте перевести его предсказания в $[0;1]$).

Отрисуйте калибровочную кривую идеально откалиброванной модели (диагональ)

In [ ]:
X, y = make_classification(
    n_samples=10000, n_features=10, n_informative=5, n_redundant=5, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
svc = SVC(probability=True)
grid_search = GridSearchCV(svc, param_grid, cv=5, verbose=2, scoring='average_precision') # 5 фолдов, average_precision - аналог pr-auc
grid_search.fit(X_train, y_train)
grid_search.predict(X_test)

In [ ]:
svc_pred_probs_test = grid_search.predict_proba(X_test)[:,1] # получаю вероятности

In [ ]:
LogReg = LogisticRegression(penalty='l2')
grid_search_lr = GridSearchCV(LogReg, param_grid, cv=5, verbose=2, scoring='average_precision') # 5 фолдов, average_precision - аналог pr-auc
grid_search_lr.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search_lr.best_params_)
print("Best Cross-Validation Score:", grid_search_lr.best_score_)

In [ ]:
y_test_pred_logreg_probs = grid_search_lr.predict_proba(X_test)[:,1] # получаю вероятности положительного класса
y_test_pred_logreg_probs

In [ ]:

plot_calibration_curve(y_test, svc_pred_probs_test, method='SVC', color1 = 'purple')


#plot_calibration_curve(y_test, y_test_pred_logreg_probs)

In [ ]:
plot_calibration_curve(y_test, y_test_pred_logreg_probs, method='Logistic Regression', color1 = 'orange')

**Вопрос**: хорошо ли откалиброваны кривые для SVM, логистической регрессии? Подумайте, как это следует из вида кривой

**Ответ:** кривая для Логрега откалибрована лучше, чем кривая для SVM, так как она в среднем менее удалена от кривой идеально откалиброванной модели (диагональ).\
Это ожидаемо, так как Логрег концентрируется на получении верных вероятностей, а SVM - на верном прогнозе класса

Из формальных способов в этом убедиться есть знакомый вам LogLoss, который напрямую оценивает вероятности,
$$\text{LogLoss} = -\frac{1}{N}\sum_{i} \sum_{k \in {0. 1}}\log p_k[y_i = k]$$
а так же BrierScore, который подсчитывает отклонение между получившейся вероятностью и реальным значением таргета.
$$\text{BrierScore} = \frac{1}{N}\sum_{i} (p_i - y_i)^2$$
Посмотрите на них тоже и сделайте вывод

In [ ]:
from sklearn.metrics import log_loss

logloss_svc = log_loss(y_test, svc_pred_probs_test)
logloss_logreg = log_loss(y_test, y_test_pred_logreg_probs)

round(logloss_svc, 4), round(logloss_logreg, 4)

Согласно LogLoss SVC лучше откалибрована, несмотря на кажущееся преимущество логрега по визуализациям кривых.\
Это нессответствие происходит потому, что LogLoss оцениает индивидуально калибровку каждого объекта, а кривая по батчам.    

In [ ]:
from sklearn.metrics import brier_score_loss

brier_svc = brier_score_loss(y_test, svc_pred_probs_test)
brier_lr = brier_score_loss(y_test, y_test_pred_logreg_probs)

round(brier_svc, 4), round(brier_lr, 4)

In [ ]:
df_metrics_calibr = pd.DataFrame(
    columns=['Logloss', 'Brier Score', 'Количество фолдов', 'Метод калибровки']
)

df_metrics_calibr.loc['SVC '] = [
      round(logloss_svc, 4),
      round(brier_svc, 4),
      5,
      ' '
]

df_metrics_calibr.loc['Log Reg '] = [
      round(logloss_logreg, 4),
      round(brier_lr, 4),
      5,
      ' '
]

df_metrics_calibr

Исходя из Brier Score у SVC отличная калибровка, а Логрег требует дополнительной калибровки

Изучите распределение ответов классификаторов при помощи гистограмм

In [ ]:
import seaborn as sns
plt.figure(figsize=(12, 8), dpi=100)
sns.kdeplot(
    data=svc_pred_probs_test,
    color="purple",
    label="Оцененные с помощью SVC вероятности",
    linewidth=2,
    fill=True,
    alpha=0.3,
)

sns.kdeplot(
    data=y_test_pred_logreg_probs,
    color="orange",
    label="Оцененные с помощью Логистической регрессии вероятности",
    linewidth=2,
    fill=True,
    alpha=0.3,
)

plt.title("Сравнение распределения ответов классификаторов",fontsize=16, pad=20)
plt.xlabel("Оцененная вероятность принадлежности к положительному классу")
plt.ylabel("Частота")
plt.legend(loc="upper right")
plt.grid(True, linestyle="--", alpha=0.7)

plt.show()

**Вопрос:** Чем они различаются? Чем вы можете объяснить это?

**Ответ:** у ответов SVC бимодальное распределение, оно более уверенное. У ответов Логрега распределение похоже на равномерное, оно более осторожное с точки зрения оценивания неопределенности.

Воспользуйтесь `CalibratedClassifierCV` из `sklearn` для калибровки вероятностей метода опорных векторов на обучении и постройте с его помощью  предсказания для тестовой выборки.

**Вопрос:** Улучшились ли калибровочная кривая и качество калибровки?

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
# я решил применить трехчастную кросс-валидацию
calibrated_svc = CalibratedClassifierCV(
    estimator=SVC(probability=False, random_state=42),
    method='sigmoid',
    cv=3  
)

calibrated_svc.fit(X_train, y_train) # обучаю на трейне

calibrated_probs = calibrated_svc.predict_proba(X_test) # предсказываю на тесте

In [ ]:
calibrated_probs[:,1][:10]

In [ ]:
plot_calibration_curve(y_test, calibrated_probs[:,1], method='откалиброванного SVC', color1 = 'purple')


**Ответ:** **Калибровочная кривая улучшилась!**

**Посмотрим на качество калибровки:**

In [ ]:
logloss_svc_calibrated = log_loss(y_test, calibrated_probs[:,1])

brier_svc_calibrated = brier_score_loss(y_test, calibrated_probs[:,1])

round(logloss_svc_calibrated, 4), round(brier_svc_calibrated, 4)

In [ ]:
df_metrics_calibr.loc['SVC с калибровкой'] = [
      round(logloss_svc_calibrated, 4),
      round(brier_svc_calibrated, 4),
      3,
      'sigmoid'
]

df_metrics_calibr

По Brier score и logloss нет улучшшения. Попробую подобрать **оптимальное количество фолдов и вид калибровки**:

In [ ]:
#methods = ['sigmoid', 'isotonic']
#cvs = [3, 5]

param_grid_calibrated_svc = {
    'method': ['sigmoid', 'isotonic'],
    'cv': [3, 5]
}


calibrated_svc = CalibratedClassifierCV(
    estimator=SVC(probability=False, random_state=42),
  #  method='sigmoid',
 #   cv=3  
)

grid_search_calibrated_svc = GridSearchCV(calibrated_svc, param_grid_calibrated_svc, verbose=2, scoring='average_precision') 
grid_search_calibrated_svc.fit(X_train, y_train)

#calibrated_svc.fit(X_train, y_train) # обучаю на трейне

calibrated_probs_upd = grid_search_calibrated_svc.predict_proba(X_test) # предсказываю на тесте

logloss_svc_calibrated_upd = log_loss(y_test, calibrated_probs_upd[:,1])

brier_svc_calibrated_upd = brier_score_loss(y_test, calibrated_probs_upd[:,1])

round(logloss_svc_calibrated_upd, 4), round(brier_svc_calibrated_upd, 4)

In [ ]:
print(f'Лучшими параметрами оказались: количество фолдов при кросс-валидации (cv) = {grid_search_calibrated_svc.best_params_['cv']}, метод калибровки: {grid_search_calibrated_svc.best_params_['method']}')

In [ ]:
df_metrics_calibr.loc['SVC с калибровкой с аодобранными cv и method'] = [
      round(logloss_svc_calibrated_upd, 4),
      round(brier_svc_calibrated_upd, 4),
      f'{grid_search_calibrated_svc.best_params_['cv']}',
      f' {grid_search_calibrated_svc.best_params_['method']}'
]

display(df_metrics_calibr.style
    .background_gradient(subset=['Logloss', 'Brier Score'], cmap='RdYlGn_r'))

Не получилось добиться лучшего качества калибровки, чем в исходном SVC

##### __Бонус: Авторское решение__ (0.5 балла)

Реализуйте свою функцию для калибровки вероятностей, используя любой из известных подходов. Кратко опишите ваш подход и продемонстрируйте результаты. Ключевые слова для вдохновения: `Platt`, `Isotonic`.

In [ ]:
# your code here

# Часть 2. Обработка категориальных переменных (4 балла + 1.5 бонус)

Как мы знаем, перекодировать категориальную переменную в список чисел (к примеру 1, 2, 3, ..., n) плохо, поскольку это бы задало на множестве ее значений некоторый порядок, не имеющий смысла.

В этой части мы рассмотрим два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

#### __Подготовка данных__

*(бесценный шаг)*

Разберем датасет [покупок велосипедов](https://www.kaggle.com/datasets/heeraldedhia/bike-buyers/): даны признаки покупателя, требуется предсказать, купит ли он/она велосипед



Замените пропуски в категориальных переменных на новую категорию (`'undefined'`)

Разделите признаки на 2 таблицы: категориальные и числовые признаки

In [ ]:
import kagglehub

path = kagglehub.dataset_download("heeraldedhia/bike-buyers") +  "/bike_buyers.csv"

In [ ]:
# Прочитаем датасет
df = pd.read_csv(path)
df.head()

In [ ]:
df.shape[0], df.ID.nunique()

In [ ]:
# Заменим пропуски категориальных переменных
df.isnull().sum()

In [ ]:
df[['Marital Status','Gender', 'Home Owner']] = df[['Marital Status','Gender', 'Home Owner']].fillna('undefined')

In [ ]:
# Отделим X и y
X = df.drop(['ID', 'Purchased Bike'], axis=1)
y = df['Purchased Bike']

In [ ]:
# Разделим на категориальные признаки и числовые
X_numerical = X[['Income', 'Children', 'Cars', 'Age']]
X_categorical = X[['Marital Status', 'Gender', 'Education', 'Occupation', 'Home Owner', 'Commute Distance', 'Region']]

В начале поработаем только с категориальными признаками

In [ ]:

y = np.where(y=='Yes', 1, 0) # для удобства сразу преобразовываю таргет в численный тип

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_categorical, y, test_size=0.25, random_state=777, stratify=y)

#### __Задание 3. OrdinalEncoder__  (0.5 балла)

Закодируйте категориальные признаки с помощью `OrdinalEncoder`. Посчитайте качество (в этом задании будем работать c __`AUC-PR`__) при применении логистической регрессии. Замерьте время, потребовавшееся на обучение модели, с учетом кодирования признаков.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
import time
start = time.time()

ord_encoder = OrdinalEncoder()
X_train1 = ord_encoder.fit_transform(X_train)
X_test1 = ord_encoder.fit_transform(X_test)

logreg_model = LogisticRegression(penalty='l2', verbose=2, random_state=777)
logreg_model.fit(X_train1, y_train)

y_test_pred_logreg = logreg_model.predict_proba(X_test1)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr = auc(recall_vals, precision_vals)

time1 = time.time() - start
print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr, 5)}')

print(f'{time1} секунд')

In [ ]:
# dataframe для сравнения
# моделей по метрикам
df_models = pd.DataFrame(
    columns=['auc_pr', 'time']   #, 'roc_auc_score', 'reg_const'
)
# добавление очередной строки с характеристиками метода
df_models.loc['LogReg (Ordinal Encoding)'] = [
      round(pr_auc_lr, 5),
      round(time1, 5)
    #  roc_auc_score(y_test, y_random),
     # 0,
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_models

Без подбора гиперпараметров и использования всех признаков качество не очень хорошее. Чуть лучше случайного классификатора

#### __Задание 4. One-Hot Encoding__ (0.5 балла)



Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (в сравнении с тем, что было до кодирования). Измерьте время, потребовавшееся на кодирование категориальных признаков и обучение модели.

In [ ]:
start = time.time()

X_train2 = pd.get_dummies(X_train, drop_first=True)
X_test2 = pd.get_dummies(X_test, drop_first=True)

logreg_model = LogisticRegression(penalty='l2', verbose=2, random_state=777)
logreg_model.fit(X_train2, y_train)

y_test_pred_logreg = logreg_model.predict_proba(X_test2)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr2 = auc(recall_vals, precision_vals)

#print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr2, 5)}')

time2 = round(time.time() - start, 5)
print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr, 5)}')
print(f'{round(time2, 5)} секунд')

In [ ]:
df_models.loc['LogReg (OHE)'] = [
      round(pr_auc_lr2, 5),
      time2
    #  roc_auc_score(y_test, y_random),
     # 0,
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_models

Как можно заметить, one-hot-кодирование может сильно увеличивать количество признаков. Это сказывается на объеме необходимой памяти, особенно, если некоторый признак имеет большое количество значений.


#### __Задание 5. Mean-target Encoding__ (1 балл)

> Проблемы разрастания числа признаков можно избежать в другом способе кодирования категориальных признаков — mean-target encoding (для простоты будем называть это __счётчиками__). Сравним эффективность методов в рамках нашей маркетинговой задачи.

> Основная идея в том, что важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:

$$
g_j(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)]}
$$

Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше, без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве.

Сравните время обучения с предыдущими экспериментами (с учетом кодирования признаков).

In [ ]:
# здесь для удобства добавляю таргет как столбец
Xtc = X_train.copy()
Xtc['buy'] = y_train
Xtc.head()

Я написал 2 реализации функции, выполняющей Mean-target Encoding. **Первая работает за O(n^2)**, при большом датасете это долго.\
**Вторая работает за O(n)**    

##### **Первая реализация**

In [ ]:
def mean_target_encoding(cat_features_df, target): # на вход подается датасет с категориальными фичами и таргет
    for col in cat_features_df.loc[:, cat_features_df.columns != target].columns:
        for category in cat_features_df[col].unique():
            cat_features_df.loc[cat_features_df[col] == category, col] = cat_features_df[cat_features_df[col] == category][target].mean()

    return cat_features_df      



In [ ]:
mean_target_encoding(Xtc, 'buy')

In [ ]:
Xtc.head(3)

#### **Вторая реализация (быстрее)**

In [ ]:
Xtc2 = X_train.copy()
Xtc2['buy'] = y_train
Xtc2.head(3)

In [ ]:
def mean_target_encoding2(cat_features_df, target): # на вход подается датасет с категориальными фичами и таргет
    for col in cat_features_df.loc[:, cat_features_df.columns != target].columns:
        cat_features_df[col] = cat_features_df.groupby(col)[target].transform('mean')

    return cat_features_df      

In [ ]:
mean_target_encoding2(Xtc2, 'buy')

In [ ]:
Xtc2.head(3)

In [ ]:
# фкнкция для применения маппинга на тестовую выборку

def apply_encoding_to_test(X_test, X_train_encoded, target_col):
    X_test_encoded = X_test.copy()
    for col in X_test.columns:
        # Беру маппинг из тренировочных данных
        mapping = X_train_encoded.groupby(col)[target_col].mean()
        X_test_encoded[col] = X_test[col].map(mapping)
    return X_test_encoded

##### **Кодирование + обучение + тест**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_categorical, y, test_size=0.25, random_state=777, stratify=y)

Xtrc = X_train.copy()
Xtrc['buy'] = y_train

Xtec = X_test.copy()



start = time.time()


Xtrc_encoded = mean_target_encoding2(Xtrc, 'buy')

Xtec_encoded = apply_encoding_to_test(Xtec, Xtrc_encoded, 'buy')  # применяю к тестовой выборке
Xtec_encoded = Xtec_encoded.fillna(Xtrc_encoded['buy'].mean())

Xtrc_final = Xtrc_encoded.drop('buy', axis=1) # удаляю таргет из трейн выборки перед обучением



logreg_model = LogisticRegression(penalty='l2', verbose=2, random_state=777)
logreg_model.fit(Xtrc_final, y_train)

y_test_pred_logreg = logreg_model.predict_proba(Xtec_encoded)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr3 = auc(recall_vals, precision_vals)

time3 = round(time.time() - start, 5)
print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr3, 5)}')

print(f'{time3} секунд')

In [ ]:
df_models.loc['LogReg (Mean-target-encoding)'] = [
      round(pr_auc_lr3, 5),
      time3
 
]

df_models

display(df_models.style
    .background_gradient(subset=['auc_pr'], cmap='RdYlGn'))

**Mean-target-encoding оказался самым эффективным способом**, при этом он работает в ~5 раз дольше, что для больших данных может быть критично

##### __Бонус: Эффективная реализация (1 балл)__

Здесь и далее реализуйте вычисление счетчиков с помощью трансформера (наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`). Обратите внимание, что все вычисления должны быть векторизованными, трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и изменять только их.

In [ ]:
# your code here

_______

__Методы борьбы с переобучением счетчиков__


Отметим, что mean-target encoding признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к __переобучению__, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его __целевая метка не использовалась__.

Это можно делать следующими способами:
1. Вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени).
2. Вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации).
3. Внесение некоторого шума в посчитанные признаки.

#### __Задание 6. Пошумим__  (0.5 балла)

Реализуйте корректное вычисление счётчиков самым простым способом — добавление шума к значениям.  При этом постарайтесь найти баланс между борьбой с переобучением и сохранением полезности признаков. Снова обучите логистическую регрессию, оцените качество.

Для начала я хочу посмотреть на разницу AUC-ROC на трейн и тест выборке прошлой модели, чтоб понять, **насколько сильное переобучение сейчас**

In [ ]:
Xtrc_encoded_fs = Xtrc_encoded.drop(columns=['buy'])

In [ ]:
print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr3, 5)}')

y_train_pred_logreg = logreg_model.predict_proba(Xtrc_encoded_fs)[:,1]    # предсказываю для трейна

precision_vals, recall_vals, thresholds = precision_recall_curve(y_train, y_train_pred_logreg)
pr_auc_lr3_train = auc(recall_vals, precision_vals)

print(f'На обучающей выборке AUC-PR = {round(pr_auc_lr3_train, 5)}')

print(f'Абсолютная разница AUC-PR = {round(pr_auc_lr3_train, 5) - round(pr_auc_lr3, 5)}')


Нетипичная картина, на тестовой выборке качество лучше

**Я добавляю шум из нормального распределения  с разными дисперсиями**

#### Функция MTE с шумом

In [ ]:
def mean_target_encoding_with_noise(cat_features_df, target, noise_level=0.05, random_state=777):
    
    np.random.seed(random_state)
    df_encoded = cat_features_df.copy()
    
    for col in cat_features_df.loc[:, cat_features_df.columns != target].columns:
        means = cat_features_df.groupby(col)[target].mean()
        
        # Применяю кодировку
        df_encoded[col] = cat_features_df[col].map(means)
        
        # Добавляю шум к обучающим данным
        # Шум из нормального распределения с std = noise_level * std таргета
        target_std = cat_features_df[target].std()
        noise = np.random.normal(0, noise_level * target_std, size=len(df_encoded))
        df_encoded[col] += noise
    
    return df_encoded

#### Обучаю модель

Пробую разные уровни шума

In [ ]:
for noise_level in tqdm([0.01, 0.03, 0.05, 0.08, 0.1, 0.15, 0.2, 0.3]):

    X_train, X_test, y_train, y_test = train_test_split(X_categorical, y, test_size=0.25, random_state=777, stratify=y)

    Xtrc = X_train.copy()
    Xtrc['buy'] = y_train
    Xtec = X_test.copy()

    start = time.time()


    Xtrc_encoded = mean_target_encoding_with_noise(Xtrc, 'buy', noise_level=noise_level, random_state=777)

    # К тестовой выборке применяю обычную кодировку без шума
    Xtec_encoded = apply_encoding_to_test(Xtec, Xtrc_encoded, 'buy')
    Xtec_encoded = Xtec_encoded.fillna(Xtrc_encoded['buy'].mean())

    Xtrc_final = Xtrc_encoded.drop('buy', axis=1)

    logreg_model = LogisticRegression(penalty='l2', verbose=2, random_state=777)
    logreg_model.fit(Xtrc_final, y_train)

    y_test_pred_logreg = logreg_model.predict_proba(Xtec_encoded)[:,1]

    precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
    pr_auc_lr4 = auc(recall_vals, precision_vals)

    time4 = round(time.time() - start, 5)
    print(f'Уровень шума = {noise_level}')
    print(f'На тестовой выборке AUC-PR = {round(pr_auc_lr4, 5)}')
    print(f'{time4} секунд')

Поднять ROC - AUC больше 0.74 не вышло, но мы сохранили результат, **это уже хорошо** 

Выберем из моделей, дающих roc-auc = 0.74 модель с **наименьшей разницей между трейн и тест выборкой**

In [ ]:
for noise_level in tqdm([0.05, 0.15, 0.3]):
    X_train, X_test, y_train, y_test = train_test_split(X_categorical, y, test_size=0.25, random_state=777, stratify=y)

    Xtrc = X_train.copy()
    Xtrc['buy'] = y_train
    Xtec = X_test.copy()

    start = time.time()


    Xtrc_encoded = mean_target_encoding_with_noise(Xtrc, 'buy', noise_level=noise_level, random_state=777)

    # К тестовой выборке применяю обычную кодировку без шума
    Xtec_encoded = apply_encoding_to_test(Xtec, Xtrc_encoded, 'buy')
    Xtec_encoded = Xtec_encoded.fillna(Xtrc_encoded['buy'].mean())

    Xtrc_final = Xtrc_encoded.drop('buy', axis=1)

    logreg_model = LogisticRegression(penalty='l2', verbose=2, random_state=777)
    logreg_model.fit(Xtrc_final, y_train)

    y_test_pred_logreg = logreg_model.predict_proba(Xtec_encoded)[:,1]

    precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
    pr_auc_lr4 = auc(recall_vals, precision_vals)

    
    y_train_pred_logreg = logreg_model.predict_proba(Xtrc_final)[:,1]

    precision_vals_tr, recall_vals_tr, thresholds_tr = precision_recall_curve(y_train, y_train_pred_logreg)
    pr_auc_lr4_train = auc(recall_vals_tr, precision_vals_tr)

    time4 = round(time.time() - start, 5)
    print(f'Уровень шума = {noise_level}')
    print(f'Разгица ROC-AUC между трейн и тест выборкой = {round(pr_auc_lr4_train - pr_auc_lr4, 5)}')
    print(f'{time4} секунд')

In [ ]:
X_train.shape

Наименьшая разница при уровне шума = **0.05**

**Вопрос:** Сделайте выводы. Помогло ли добавление шума? Почему?

**Ответ:** Добавление шума не помогло. Я связываю это с достаточно большим кол-вом данных, плюс, видимо, переобучения не было изначально

##### __Бонус: другой подход__ (0.5 балла)

Посчитайте корректные счётчики первым или вторым способов из описанных выше (не забудьте добавить и шум).




In [ ]:
# your code here

#### __Задание 7. Сглаживание счетчиков__  (1 балл)

> Теперь ответим на следующий вопрос: что будет, если некоторая категория встречается в выборке всего несколько раз? По этой причине производится сглаживание счётчиков. Например, на практике хорошие результаты показывает использование сглаживания средним по всей выборке:
$$
g_j(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = +1] + C \times \text{global_mean}}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)] + C}
$$
где $\text{global_mean}$ — доля объектов положительного класса в выборке, $C$ — параметр, определяющий степень сглаживания (можно использовать 10 или подобрать для каждого признака свой). Идея в том, что мы "разбавляем" среднее значение по категории глобальным средним значением. И тем меньше, чем большее количество объектов этой категории встречается в выборке.

> Вместо среднего значения целевой переменной для сглаживания можно использовать любое другое значение от 0 до 1 (этот параметр иногда называют $prior$). Можно сделать несколько признаков с разными значениями параметра. На практике в задачах бинарной классификации полезными бывают даже отрицательные значения!

Добавьте сглаживание, описанное выше и повторите эксперименты. Подберите $C$, чтобы качество было лучше, чем при использовании One-Hot-Encoding


In [ ]:
# your code here

#### **Задание 8. Числовые или категориальные?**  (0.5 балла)

Теперь добавим числовые признаки к счётчикам (тем, которые дали наибольший прирост качества).


Проверьте их на наличие выбросов и заполните пропуски средним или медианой, подумайте, что лучше в условиях наших данных



### EDA числовых признаков

In [ ]:
X_numerical.head()

In [ ]:
sns.histplot(X_numerical['Income'], bins=30, kde=True, color='#4C72B0', edgecolor='red')

plt.axvline(X_numerical['Income'].median(), color='yellow', linestyle='--', linewidth=2, label=f'Медиана: {X_numerical['Income'].median():.1f}')
plt.axvline(X_numerical['Income'].mean(), color='red', linestyle='--', linewidth=2, label=f'Среднее: {X_numerical['Income'].mean():.1f}')

plt.xlabel('Доход')
plt.ylabel('Частота')
plt.title('Гистограмма распределения по доходу')
plt.legend()
plt.show()

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10, 2))
boxplot = sns.boxplot(
    data=X_numerical['Income'], 
    orient="h",
    width=0.6,
    color="#E84393",
    linewidth=2,
    flierprops=dict(
        markerfacecolor='#DD8452',
        marker='D', 
        markersize=6
    ),
    boxprops=dict(alpha=0.8)
)
plt.title("Распределение доходов людей", pad=20, fontsize=14)
plt.xlabel("Доход", fontsize=12)
#plt.yticks([])
plt.show()

In [ ]:
X_numerical[X_numerical['Income'] > 140000].shape[0]

In [ ]:
# Выбросов немного, оставлю их.

In [ ]:
X_numerical['Income'].unique()

Выбросов не очень много. Я предполагаю, что распределение логнормальное. Проверим это:

In [ ]:
sns.histplot(np.log1p(X_numerical['Income']), bins=30, kde=True, color='#4C72B0', edgecolor='red')
plt.xlabel('Доход')
plt.ylabel('Частота')
plt.title('Гистограмма распределения по доходу')
plt.legend()
plt.show()

Не уверен, что логнормальное

In [ ]:
X_numerical.isnull().sum()

In [ ]:
#Заполняю пропуски в доходе медианой, их немного

X_numerical['Income'].fillna(X_numerical['Income'].median(), inplace=True)

In [ ]:
sns.histplot(X_numerical['Age'], bins=30, kde=True, color='orange', edgecolor='red')

plt.axvline(X_numerical['Age'].median(), color='yellow', linestyle='--', linewidth=2, label=f'Медиана: {X_numerical['Age'].median():.1f}')
plt.axvline(X_numerical['Age'].mean(), color='red', linestyle='--', linewidth=2, label=f'Среднее: {X_numerical['Age'].mean():.1f}')

plt.xlabel('Возраст')
plt.ylabel('Частота')
plt.title('Гистограмма распределения по возрасту')
plt.legend()
plt.show()

Судя по гистограмме, есть выбросы

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(10, 2))
boxplot = sns.boxplot(
    data=X_numerical['Age'], 
    orient="h",
    width=0.6,
    color="orange",
    linewidth=2,
    flierprops=dict(
        markerfacecolor='#DD8452',
        marker='D', 
        markersize=6
    ),
    boxprops=dict(alpha=0.8)
)
plt.title("Распределение возраста людей", pad=20, fontsize=14)
plt.xlabel("Возраст", fontsize=12)
#plt.yticks([])
plt.show()

In [ ]:
X_numerical[X_numerical['Age'] > 75].shape[0]

Выбросов по возрасту почти нет. Оставляем как есть.

In [ ]:
# пропуски заменяю медианой

X_numerical['Age'].fillna(X_numerical['Age'].median(), inplace=True)

In [ ]:
plt.figure(figsize=(8, 5))
sns.set_theme(style="whitegrid")

ax = sns.countplot(
    data=X_numerical,
    x='Children',

    edgecolor='black'
)

for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3)

plt.title('Разбивка выборки по количеству детей', fontsize=14, pad=10)
plt.xlabel('Количество детей', fontsize=13)
plt.ylabel('Частота', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
X_numerical['Children'].median()

In [ ]:
#Заменим пропуски медианой - 2 ребенка, как ине кажется, это достаточно правдоподобно

X_numerical['Children'].fillna(X_numerical['Children'].median(), inplace=True)

In [ ]:
plt.figure(figsize=(8, 5))
sns.set_theme(style="whitegrid")

ax = sns.countplot(
    data=X_numerical,
    x='Cars',
    edgecolor='black'
)

for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3)

plt.title('Разбивка выборки по количеству машин', fontsize=14, pad=10)
plt.xlabel('Количество машин', fontsize=13)
plt.ylabel('Частота', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
X_numerical['Cars'].median()

In [ ]:
# Заменим пропуски на медиану - 1 машину

X_numerical['Cars'].fillna(X_numerical['Cars'].median(), inplace=True)

 Сейчас для числовых признаков мы ищем линейную зависимость, что в общем случае  может быть неверной гипотезой. Тем не менее, у этих признаков есть довольно много уникальных значений (сколько?), поэтому применять к ним one-hot кодирование может оказаться излишним. Попробуйте закодировать эти признаки с помощью счетчиков. Стало ли лучше?

In [ ]:
for ft in X_numerical.columns:
    print(f'У признака {ft} {X_numerical[ft].nunique()} уникальных значений')

In [ ]:
X8 = pd.merge(X_numerical, X_categorical, left_index=True, right_index=True)

In [ ]:
X8.head()

Кодирую с помощью счетчиков:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X8, y, test_size=0.25, random_state=777, stratify=y)

Xtrc = X_train.copy()
Xtrc['buy'] = y_train

Xtec = X_test.copy()



start = time.time()


Xtrc_encoded = mean_target_encoding2(Xtrc, 'buy')

Xtec_encoded = apply_encoding_to_test(Xtec, Xtrc_encoded, 'buy')  # применяю к тестовой выборке
Xtec_encoded = Xtec_encoded.fillna(Xtrc_encoded['buy'].mean())

Xtrc_final = Xtrc_encoded.drop('buy', axis=1) # удаляю таргет из трейн выборки перед обучением



logreg_model = LogisticRegression(penalty='l2', verbose=2, random_state=777)
logreg_model.fit(Xtrc_final, y_train)

y_test_pred_logreg = logreg_model.predict_proba(Xtec_encoded)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr8 = auc(recall_vals, precision_vals)

time8 = round(time.time() - start, 5)

print(f'На тестовой выборке AUC-ROC = {round(pr_auc_lr8, 5)}')

print(f'{time8} секунд')

**Лучше почему-то не стало. Пока не можем пробить ROC-AUC больше 0.74**

> __Замечание.__ Усложнение методов вычисления счётчиков не делают результаты модели гарантированно лучше. Особенно с учётом того, что логистическая регрессия не такая сложная модель, чтобы переобучаться. Поэтому вы необязательно должны были получать на каждом шаге всё лучшие и лучшие результаты (но необходимые результаты у вас должны были получиться).



Как мы могли пронаблюдать, счётчики являются конкурентной альтернативой one-hot-кодированию. Опишите, какие плюсы и минусы использования счётчиков по сравнению с one-hot-кодированием вы заметили.

__Ответ:__ 

**Плюсы MTE в сравнении с OHE**: не увеличвает размерность, сохраняет инф-ю о таргете, лучше работает с редкими категориями, можно проще интерпретировать

**Минусы MTE в сравнении с OHE**: утечка таргета в признаки, высокий риск переобучения, сложно обработать новые категории в тесте

# Часть 3. Отбор признаков (2 балла)

Загрузим данные [UCI Adult Dataset](https://archive.ics.uci.edu/ml/datasets/Adult). Этот набор данных содержит информацию о годовых доходах отдельных людей. В качестве признакового описания используется различная информация о человеке (образование, профессия, брачный статус и т.д.). Целевая переменная является бинарной: больше ли годовой доход 50K долларов или нет.

In [ ]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [ ]:
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
    'income'
]

df = pd.read_csv('adult.data', header=None, names=columns)
df['income'] = (df['income'] != " <=50K").astype('int32')
df.sample()

In [ ]:
df.shape

In [ ]:
df['fnlwgt'].unique()

Я прочитал, что **fnlwgt отражает количество людей со схожими характеристиками**. Поэтому при прогнозировании использовать его я не буду.

In [ ]:
df.isnull().sum()

In [ ]:
df['occupation'].value_counts()

Важной частью процесса построения модели является отбор признаков. На практике многие признаки оказывают малое влияние на модель (при этом их расчёт занимает время) или даже негативно сказываются на качестве модели. Попробуем несколько подходов отбора признаков, оценим, как они влияют на качество модели и сколько времени занимают.



Разделите выборку на обучающую и тестовую в соотношении 3:1. Зафиксируйте `random_state=777`, также используйте `stratify=True`.

In [ ]:
y = df['income']
X = df.drop(columns=['fnlwgt', 'income', 'education'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

Давайте закодируем все категориальные признаки с помощью One-hot Encoding. Сколько новых признаков мы получим?

In [ ]:
X_train.columns

**education-num** - это числовое представление признака education. Я оставлю его, и удалю educztion, так как education-num сохраняет порядок категорий, что для признака уровень образования и наших классификаторов (SVM, Лог регрессия) будет полезно (удаляю education выше)

In [ ]:
X_train['workclass'].value_counts()

In [ ]:
X_train['native-country'].value_counts()[:3]

**Числовые признаки я решил обработать Scalerом**

In [ ]:
# подготовка признаков

categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']  # категориальные признаки
numeric_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'] # числовые признаки

preprocessor1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])


preprocessor1.fit(X_train)
X_train_transformed = preprocessor1.transform(X_train)
X_test_transformed = preprocessor1.transform(X_test)

In [ ]:
print(f"Исходная размерность X_train: {X_train.shape}")
print(f"Размерность после OHE и Scaling: {X_train_transformed.shape}")
print(f"Количество новых признаков: {X_train_transformed.shape[1] - X_train.shape[1]}")

В качестве основной модели будем использовать логистическую регрессию, а целевой метрики — `AUC-PR`. Обучите модель и посчитайте качество на тестовой выборке. Давайте запомним полученное значение.

In [ ]:
# подбираю оптимальное C

param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

In [ ]:
LogReg = LogisticRegression(penalty='l2')
grid_search_lr1 = GridSearchCV(LogReg, param_grid, cv = 5, verbose=2, scoring='average_precision') # 5 фолдов для кросс-валидации
grid_search_lr1.fit(X_train_transformed, y_train)


y_test_pred_logreg = grid_search_lr1.predict_proba(X_test_transformed)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr9 = auc(recall_vals, precision_vals)


print("Best Hyperparameters:", grid_search_lr1.best_params_)
print("Best Cross-Validation Score:", grid_search_lr1.best_score_)
print("Test Score:", pr_auc_lr9)

In [ ]:
df_metrics_part3 = pd.DataFrame(
    columns=['auc_pr_valid', 'auc_pr_test', 'reg_const']
)
#precision, recall, _ = precision_recall_curve(y_test, y_random)
# добавление очередной строки с характеристиками метода
df_metrics_part3.loc['LogReg'] = [
      round(grid_search_lr1.best_score_, 4),
      round(pr_auc_lr9, 4),
      grid_search_lr1.best_params_['C']
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_metrics_part3

**Качество на тесте неплохое для такой простой модели!**

Допустим, мы хотим оставить только 40 лучших признаков.

Заметим, что нельзя оценивать качество по тестовой выборке, иначе мы можем переобучиться, как, например, при настройке гиперпараметров. Разделите обучающую выборку на 2 части, одну из которых, используйте для валидации. Исходную тестовую выборку стоит использовать только для финальной оценки качества после процедуры фильтрации

Я прочитал, что для отбора признаков очень хорош **случайный лес, так как в нем встроен отбор признаков, + он улавливает сложные взаимодействия между признаками**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# готовлю данные


y = df['income']
X = df.drop(columns=['fnlwgt', 'income', 'education'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.35, random_state=777) # я решил оставить 35% на валидацию


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])

In [ ]:
# Реализовываю пайплайн

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selector', SelectFromModel(
        RandomForestClassifier(n_estimators=100, random_state=777, n_jobs=-1),
        max_features=40
    )),
    ('classifier', LogisticRegression(penalty='l2', random_state=777, max_iter=1000))
])

# сетка гиперпараметров
param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['liblinear', 'lbfgs'] # решил добавить еще и подбор метода оптимизации
}



grid_search_lr = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5, 
    verbose=2, 
    scoring='average_precision',
    n_jobs=-1
)

grid_search_lr.fit(X_train, y_train) 


y_test_pred_logreg = grid_search_lr.predict_proba(X_test)[:, 1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg)
pr_auc_lr = auc(recall_vals, precision_vals)

print("Best Hyperparameters:", grid_search_lr.best_params_)
print("Best Cross-Validation Score:", grid_search_lr.best_score_)
print("Test PR-AUC Score:", round(pr_auc_lr, 4))


Качество на тестовой выборке **чуть упало**

In [ ]:
df_metrics_part3.loc['LogReg with 40 best features (selected by RandomForest)'] = [
      round(grid_search_lr.best_score_, 4),
      round(pr_auc_lr, 4),
      100
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку
df_metrics_part3

In [ ]:
# Визуализация важности признаков (в построении мне помог deepseek)

best_pipeline = grid_search_lr.best_estimator_
feature_selector = best_pipeline.named_steps['feature_selector']
preprocessor = best_pipeline.named_steps['preprocessor']

# имена признаков
all_feature_names = preprocessor.get_feature_names_out()

selected_mask = feature_selector.get_support()
selected_feature_names = all_feature_names[selected_mask]

# важность признаков из RF
rf_model = feature_selector.estimator_
feature_importance = rf_model.feature_importances_

selected_importance = feature_importance[selected_mask]

importance_df = pd.DataFrame({
    'feature': selected_feature_names,
    'importance': selected_importance
}).sort_values('importance', ascending=True)

plt.figure(figsize=(12, 12))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Важность признака')
plt.title('Топ самых важных признаков (Random Forest)', fontsize=14, pad=10)
plt.tight_layout()
plt.show()


Как я и предполагал, в топе важных признаков - **возраст, уровень образования**

Попробуем сделать это следующими способами:

#### __Задание 9. Встроенные методы (0.5 балла)__

Начнём с отбора признаков с помощью модели. У разных алгоритмов есть разные встроенные способы оценки вклада признаков в предсказание. Как известно, у линейной модели за это отвечают веса, а значит, их модуль можно интерпретировать как важность. Такой метод отбора называются встроенным или embedded method, так как он заложен в особенности модели.

Оставьте 40 признаков с наибольшим модулем соответствующего параметра линейной модели. Обучите модели заново и оцените её качество. Замерьте скорость такого отбора признаков.



In [ ]:
best_logreg = grid_search_lr1.best_estimator_

coefficients = best_logreg.coef_[0]


feature_names = preprocessor1.get_feature_names_out()

coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)
coef_df.head(10)

In [ ]:
coef_df.shape

In [ ]:
y_train.shape

In [ ]:
X_train_transformed.shape, X_train.shape

In [ ]:
# обучаю модель на 40 лучших признаках

top_40_features = coef_df.head(40)['feature'].values


feature_mask = [name in top_40_features for name in feature_names]

X_train_top40 = X_train_transformed[:, feature_mask]
X_test_top40 = X_test_transformed[:, feature_mask]
#y_train_top40 = y_train[feature_mask]


logreg_top40 = LogisticRegression(penalty='l2', random_state=777, max_iter=1000)
grid_search_top40 = GridSearchCV(
    logreg_top40, 
    param_grid, 
    cv=5, 
    verbose=2, 
    scoring='average_precision'
)
grid_search_top40.fit(X_train_top40, y_train)

# оцениваю на тестовой выборке
y_test_pred_top40 = grid_search_top40.predict_proba(X_test_top40)[:, 1]
precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_top40)
pr_auc_top40 = auc(recall_vals, precision_vals)

print("Best Hyperparameters:", grid_search_top40.best_params_)
print("Best Cross-Validation Score:", grid_search_top40.best_score_)
print("Test PR-AUC Score:", pr_auc_top40)

In [ ]:
df_metrics_part3.loc['LogReg with 40 best features (selected by weights)'] = [
      round(grid_search_top40.best_score_, 4),
      round(pr_auc_top40, 4),
      100000
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку

display(df_metrics_part3.style
    .background_gradient(subset=['auc_pr_test'], cmap='RdYlGn'))

In [ ]:
# здесь чуть поломалось при повторном запуске, но выводы ниже верные

Изменилось ли качество? Как?

**Качество чуть улучшилось при отборе по весам!**\
При этом отбор с помощью RandomForest показал худшие результаты на тесте

Подумаем, что мы не учли. Мы действовали в предположении, что признаки вносят вклад равномерно, и не учитывали их масштаб. Если мы умножим один из признаков в 100 раз, то без учёта регуляризации его вес уменьшится в эти же 100 раз. А мы на основе этого отбираем признаки! Давайте сначала отмасштабируем признаки одним из способов, а только потом будем удалять признаки.

Помните, что не все способы одинаково хороши, особенно в условиях наличия выбросов

Что получилось?

Я **отмасштабировал в самом начале**

Вопрос на засыпку: one-hot кодирование возвращает нам единичные признаки-индикаторы. Попробуйте также отскалировать их, как и обычные числовые, и снова выбрать 40 главных по вкладу признаков. Изменился ли их список? Изменится ли качество?

In [ ]:
y = df['income']
X = df.drop(columns=['fnlwgt', 'income', 'education'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

In [ ]:
new_pipeline = Pipeline([
    ('ohe', ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )),
    ('scaler', StandardScaler())
])

new_pipeline.fit(X_train)
X_train_scaled = new_pipeline.transform(X_train)
X_valid_scaled = new_pipeline.transform(X_valid)
X_test_scaled = new_pipeline.transform(X_test)


feature_names = new_pipeline.named_steps['ohe'].get_feature_names_out()
print("Имена признаков после OHE + StandardScaler:")
print(feature_names[:40])  # первые 40 признаков

In [ ]:
X_train_transformed.shape

In [ ]:
len(feature_mask[:-1])

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs'] # решил добавить еще и подбор метода оптимизации
}

In [ ]:
top_40_features1 = feature_names[:40]
feature_mask = [name in top_40_features1 for name in feature_names]

X_train_top40 = X_train_scaled[:, feature_mask]
X_test_top40 = X_test_scaled[:, feature_mask]


logreg_top40_ohe_ss = LogisticRegression(penalty='l2', random_state=777, max_iter=1000)
grid_search_top40_ohe_ss = GridSearchCV(
    logreg_top40_ohe_ss, 
    param_grid, 
    cv=5, 
    verbose=2, 
    scoring='average_precision'
)
grid_search_top40_ohe_ss.fit(X_train_top40, y_train)

# оцениваю на тестовой выборке
y_test_pred_top40 = grid_search_top40_ohe_ss.predict_proba(X_test_top40)[:, 1]
precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_top40)
pr_auc_top40 = auc(recall_vals, precision_vals)

print("Best Hyperparameters:", grid_search_top40_ohe_ss.best_params_)
print("Best Cross-Validation Score:", grid_search_top40_ohe_ss.best_score_)
print("Test PR-AUC Score:", pr_auc_top40)

Качество очень упало. Это связано с тем, что **не стоит использовать методы обработки числовых признаков для категориальных**

#### __Задание 10. Методы фильтрации (0.5 балла)__


Давайте отбирать признаки умнее, а именно через подсчёт некоторой функции для каждого признака. На основании значений этой функции будем оставлять наиболее важные признаки. Методы этого семейства называют фильтрующими или  filter methods.

Одна из самых простых функция - корреляция между признаком и целевой переменной. Подумайте, какая взаимосвязь между корреляцией и предсказательной способностью модели, и как бы вы использовали информацию о корреляции для отбора признаков

**Ответ:** чем выше модуль корреляции между признаком и таргетом - **тем сильнее линейная взаимосвязь между ними.** Логично предположить, что чем выше корреляция таргета и признаков, тем сильнее предсказательная способность модели. Я бы брал признаки с наибольшей по модулю корреляцией с таргетом.

Посчитайте корреляцию каждого признака с таргетом и отфильтруйте 40 признаков исходя из того, что вы описали, после чего замерьте качество и время отбора



In [ ]:
y = df['income']
X = df.drop(columns=['fnlwgt', 'income', 'education'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=777)

In [ ]:
# подготовка признаков

categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']  # категориальные признаки
numeric_features = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'] # числовые признаки

preprocessor_new = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])


preprocessor_new.fit(X_train)
X_train_transformed = preprocessor_new.transform(X_train)
X_test_transformed = preprocessor_new.transform(X_test)

In [ ]:
# сопоставляю имена со столбцами X_train_transformed

cat_encoder = preprocessor_new.named_transformers_['cat']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_features)

all_feature_names = numeric_features + list(cat_feature_names)

print(f"Всего признаков: {len(all_feature_names)}")

In [ ]:
X_train_df = pd.DataFrame(X_train_transformed, columns=all_feature_names)
X_test_df = pd.DataFrame(X_test_transformed, columns=all_feature_names)

X_train_df['target'] = y_train.values

In [ ]:
# считаю корреляции
correlations = X_train_df.corr()['target'].sort_values(ascending=False)

correlations.head(3)

In [ ]:
# беру модули

correlations_abs = np.abs(correlations)
correlations_abs.sort_values(ascending=False).head(41) # беру 40 лучших

In [ ]:
top_40_corr  = correlations_abs.sort_values(ascending=False).head(41).index.tolist()
top_40_corr = [el for el in top_40_corr if el != 'target']

In [ ]:
# обучаю только на них

X_train_corr = X_train_df.drop(columns='target', axis=1)
X_test_corr = X_test_df.copy()

X_train_corr = X_train_corr[top_40_corr]
X_test_corr = X_test_corr[top_40_corr]





In [ ]:
LogReg_top40corr = LogisticRegression(penalty='l2', max_iter=1000)
grid_search_lr0 = GridSearchCV(LogReg_top40corr, param_grid, cv = 5, verbose=2, scoring='average_precision') # 5 фолдов для кросс-валидации
grid_search_lr0.fit(X_train_corr, y_train)


y_test_pred_logreg0 = grid_search_lr0.predict_proba(X_test_corr)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg0)
pr_auc_lr0 = auc(recall_vals, precision_vals)


print("Best Hyperparameters:", grid_search_lr0.best_params_)
print("Best Cross-Validation Score:", grid_search_lr0.best_score_)
print("Test Score:", pr_auc_lr0)

In [ ]:
df_metrics_part3.loc['LogReg with 40 best features (selected by correlations with target)'] = [
      round( grid_search_lr0.best_score_, 4),
      round(pr_auc_lr0, 4),
      10
]

# по аналогии результаты следующих экспериментов можно будет собрать в табличку

display(df_metrics_part3.style
    .background_gradient(subset=['auc_pr_test'], cmap='RdYlGn'))

**Качество почти такое же, как при отборе по весам**

В качестве еще одной функция можно считать t-статистику:

$$t(j) = \frac{|\mu_+ - \mu_-|}{\sqrt{\frac{n_+ \sigma^2_+ + n_- \sigma^2_-}{n_+ + n_-}}},$$

где $\mu$, $\sigma$, $n$ соответственно среднее, стандартное отклонение и количество объектов каждого из классов.

Оставьте 40 признаков с наибольшим значением $t$, замерьте качество и скорость отбора признаков.

In [ ]:
Xt = X_train_df.copy()
Xt.head()

Я решил написать **алгоритм отбора с аомощью t-stat в виде функции**

In [ ]:
def selection_tstat(frame, target_col, n_best): 

    ft_tstat_list = []

    frame_cl = frame.drop(columns=[target_col], axis=1)
    for ft in frame_cl.columns.tolist():

        m_plus = frame[frame[target_col]==1][ft].mean()
        m_minus = frame[frame[target_col]==0][ft].mean()
        
        n_plus = frame[frame[target_col]==1].shape[0]
        n_minus = frame[frame[target_col]==0].shape[0]
        
        sigma2_plus = frame[frame[target_col]==1][ft].var()
        sigma2_minus = frame[frame[target_col]==0][ft].var()

        t_stat_j = np.abs((m_plus - m_minus)) / np.sqrt((n_plus*sigma2_plus + n_minus*sigma2_minus) / (n_plus + n_minus)) # t-stat для j-го признака

        ft_tstat_list.append({'feature': ft, 't_stat': t_stat_j})

    ft_tstat_frame = pd.DataFrame(ft_tstat_list)

    ft_tstat_frame = ft_tstat_frame.sort_values(by='t_stat', ascending=False)[:n_best] # вывожу n_best лучших


    return ft_tstat_frame
        


In [ ]:
start10 = time.time()

best_features_tstat = selection_tstat(Xt, 'target', 40)

time10 = time.time() - start10

print(f'Отбор признаков с помощью t-stat занял {round(time10, 3)} секунд')

In [ ]:
# обучаю модель на 40 лучших с точки зрения t-stat признаках

top_40_tstat = best_features_tstat['feature']

X_train_tstat = X_train_df.drop(columns='target', axis=1)
X_test_tstat = X_test_df.copy()

X_train_tstat = X_train_corr[top_40_tstat]
X_test_tstat = X_test_corr[top_40_tstat]

In [ ]:
LogReg_top40tstat = LogisticRegression(penalty='l2', max_iter=1000)
grid_search_lr000 = GridSearchCV(LogReg_top40tstat, param_grid, cv = 5, verbose=2, scoring='average_precision') # 5 фолдов для кросс-валидации
grid_search_lr000.fit(X_train_tstat, y_train)


y_test_pred_logreg000 = grid_search_lr000.predict_proba(X_test_tstat)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg000)
pr_auc_lr000 = auc(recall_vals, precision_vals)


print("Best Hyperparameters:", grid_search_lr000.best_params_)
print("Best Cross-Validation Score:", grid_search_lr000.best_score_)
print("Test Score:", pr_auc_lr000)

In [ ]:
df_metrics_part3.loc['LogReg with 40 best features (selected by t-stat)'] = [
      round( grid_search_lr000.best_score_, 4),
      round(pr_auc_lr000, 4),
      10
]

display(df_metrics_part3.style
    .background_gradient(subset=['auc_pr_test'], cmap='RdYlGn'))

**Пока не удается пробить качество выше auc_pr_test = 0.7674**

#### __Задание 11. Методы-обёртки__ (1 балл)

Третий из рассматриваемых нами методов работает следующим образом: мы исключаем признаки по очереди и смотрим, как это влияет на качество. Удаляем признаки таким жадным способом, пока не окажется выполненым некоторое условие (количество признаков или ухудшение качества). Более конкретно, алгоритм выглядит так:

- $k$ - число признаков, которых мы хотим оставить
- $m$ - число признаков, которых мы выбрасываем на каждой итерации, оно же длина шага

Шаг $i$:
- $F_i$ - набор признаков (равный всему множеству признаков на i=0)
- $M_i$ - их число, в общем случае $\max(k, M_{i-1} - m)$
1. Если признаков осталось ровно $k$, либо метрика стала уменьшаться более, чем на $\epsilon$ — останавливаемся (не наш случай, но так тоже можно)
2. Обучаем модель $a_i$ на наборе $F_i$, после чего оцениваем важность признаков (любым из способов выше или какими-нибудь ещё)
3. Отбираем $\min(M_i - k, m)$ наиболее бесполезных, согласно пункту 2, признаков (берем $m$, если можем, иначе оставляем вплоть до k), удаляем, переходим к следующему шагу

Снова оставьте только 40 признаков и оцените качество на тестовой выборке. Подберите длину шага из каких-то соображений (каких, кстати?) и замерьте время работы метода

Мне понравился **метод с t-stat**, поэтому буду использовать его

In [ ]:
X11 = X_train_df.copy()
X11.head()

In [ ]:
X_test_11 = X_test_df.copy()

In [ ]:
def selection_tstat_11(frame, target_col): 

    ft_tstat_list = []

    frame_cl = frame.drop(columns=[target_col], axis=1)
    for ft in frame_cl.columns.tolist():

        m_plus = frame[frame[target_col]==1][ft].mean()
        m_minus = frame[frame[target_col]==0][ft].mean()
        
        n_plus = frame[frame[target_col]==1].shape[0]
        n_minus = frame[frame[target_col]==0].shape[0]
        
        sigma2_plus = frame[frame[target_col]==1][ft].var()
        sigma2_minus = frame[frame[target_col]==0][ft].var()

        t_stat_j = np.abs((m_plus - m_minus)) / np.sqrt((n_plus*sigma2_plus + n_minus*sigma2_minus) / (n_plus + n_minus)) # t-stat для j-го признака

        ft_tstat_list.append({'feature': ft, 't_stat': t_stat_j})

    ft_tstat_frame = pd.DataFrame(ft_tstat_list)

    ft_tstat_frame = ft_tstat_frame.sort_values(by='t_stat', ascending=False)                # возвращаю таблицу признаков по убыванию важности согласно t-stat


    return ft_tstat_frame
        

In [ ]:
def wrapper(frame_train):
    k = frame_train.shape[1] - 1

    pbar = tqdm(total=k-40)  # для отслеживания прогресса

    frame_train_upd = frame_train.drop('target', axis=1)

    current_features = frame_train_upd.columns.tolist()

    while k > 40:

        ft_tstat_frame_11 = selection_tstat_11(frame_train[current_features + ['target']], 'target')

        features_step_i = ft_tstat_frame_11['feature'].tolist()[:-1]      # на каждом шаге выкидываю по 1 самому худшему признаку

        current_features = features_step_i

        k -= 1
        pbar.update(1) 

    pbar.close()    

    return features_step_i 

In [ ]:
start11 = time.time()


features_selected_by_wrapper = wrapper(X11)

time11 = time.time() - start11
print(f'Отбор признаков с помощью метода обертки занял {round(time11, 3)} секунд')

In [ ]:
features_selected_by_wrapper[:2]

In [ ]:
# обучаю модель на отобранных признаках и смотрю качество

X11 = X11.drop(columns='target', axis=1)

X_train_11 = X11[features_selected_by_wrapper]
X_test_11 = X_test_11[features_selected_by_wrapper]



LogReg_top40wrapper = LogisticRegression(penalty='l2', max_iter=1000)
grid_search_lr111 = GridSearchCV(LogReg_top40wrapper, param_grid, cv = 5, verbose=2, scoring='average_precision') # 5 фолдов для кросс-валидации
grid_search_lr111.fit(X_train_11, y_train)


y_test_pred_logreg111 = grid_search_lr111.predict_proba(X_test_11)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg111)
pr_auc_lr111 = auc(recall_vals, precision_vals)


print("Best Hyperparameters:", grid_search_lr111.best_params_)
print("Best Cross-Validation Score:", grid_search_lr111.best_score_)
print("Test Score:", pr_auc_lr111)


In [ ]:
df_metrics_part3.loc['LogReg with 40 best features (selected by wrapper)'] = [
      round( grid_search_lr111.best_score_, 4),
      round(pr_auc_lr111, 4),
      10
]

display(df_metrics_part3.style
    .background_gradient(subset=['auc_pr_test'], cmap='RdYlGn'))

In [ ]:
len(set(top_40_tstat) & set(features_selected_by_wrapper))

Получается, отбор с помощью t-stat и отбор с помощью t-stat несколькими итерациями дают **одинаковый набор признаков**

Стоит отметить, что с помощью такого метода можно пойти и в обратную сторону. Попробуйте _добавлять_ самые полезные признаки в выборку до тех пор, пока не наберется 40 штук. Найдется ли порог, при котором добавление следующих признаков будет только ухудшать качество модели?

In [ ]:
X12 = X_train_df.copy()
X_test_12 = X_test_df.copy()

In [ ]:
# реализовываю алгоритм с добавлением признаков

def wrapper_reversed(frame_train):
    k = 1

    pbar = tqdm(total=k-40)  # для отслеживания прогресса

    frame_train_upd = frame_train.drop('target', axis=1)

    current_features = frame_train_upd.columns.tolist()

    ft_tstat_frame_11 = selection_tstat_11(frame_train[current_features + ['target']], 'target')

    while k <= 40:

        

        features_step_j = ft_tstat_frame_11['feature'].tolist()[:k]      # на каждом шаге добавляю по 1 признаку

        current_features = features_step_j

        k += 1
        pbar.update(1) 

    pbar.close()    

    return features_step_j

In [ ]:
start12 = time.time()


features_selected_by_reversed_wrapper = wrapper_reversed(X12)

time12 = time.time() - start12
print(f'Отбор признаков с помощью обратного метода обертки занял {round(time12, 3)} секунд')

**Намного быстрее!**

In [ ]:
# обучаю модель на отобранных признаках и смотрю качество

X12 = X12.drop(columns='target', axis=1)

X_train_12 = X12[features_selected_by_reversed_wrapper]
X_test_12 = X_test_12[features_selected_by_reversed_wrapper]



LogReg_top40wrapper_rev = LogisticRegression(penalty='l2', max_iter=1000)
grid_search_lr1112 = GridSearchCV(LogReg_top40wrapper_rev, param_grid, cv = 5, verbose=2, scoring='average_precision') # 5 фолдов для кросс-валидации
grid_search_lr1112.fit(X_train_12, y_train)


y_test_pred_logreg1112 = grid_search_lr1112.predict_proba(X_test_12)[:,1]

precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred_logreg1112)
pr_auc_lr1112 = auc(recall_vals, precision_vals)


print("Best Hyperparameters:", grid_search_lr1112.best_params_)
print("Best Cross-Validation Score:", grid_search_lr1112.best_score_)
print("Test Score:", pr_auc_lr1112)

In [ ]:
df_metrics_part3.loc['LogReg with 40 best features (selected by reversed wrapper)'] = [
      round( grid_search_lr1112.best_score_, 4),
      round(pr_auc_lr1112, 4),
      10
]

display(df_metrics_part3.style
    .background_gradient(subset=['auc_pr_test'], cmap='RdYlGn'))

Ожидаемо, **качество такое же**

Построим **визуализацию динамики качества на тестовой выборке** с добавлением новых признаков

In [ ]:
X13 = X_train_df.copy()
X_test_13 = X_test_df.copy()

Я строю модель без подбора гиперпараметров GridSearch, так как иначе **вычисления слишком долгие**

In [ ]:
k = 1
arr_k = [k]

ft_tstat_frame_11 = selection_tstat_11(X13, 'target')
sorted_features = ft_tstat_frame_11['feature'].tolist()

aucs = [0]
X_test_13_full = X_test_13[sorted_features]

total_iterations = X13.shape[1] - 1 - k
pbar = tqdm(total=total_iterations)

simple_logreg = LogisticRegression(penalty='l2', C=1.0, max_iter=1000)

while k < X13.shape[1] - 1:
    current_features = sorted_features[:k]
    
    X_train_13 = X13[current_features]
    X_test_13 = X_test_13_full[current_features]
    
    simple_logreg.fit(X_train_13, y_train)
    y_test_pred = simple_logreg.predict_proba(X_test_13)[:,1]
    
    precision_vals, recall_vals, thresholds = precision_recall_curve(y_test, y_test_pred)
    pr_auc_lr1112 = auc(recall_vals, precision_vals)
    
    k += 1
    arr_k.append(k)
    aucs.append(pr_auc_lr1112)
    pbar.update(1)

pbar.close()
print('Готово')

In [ ]:
len(arr_k), len(aucs)

Строю график:

In [ ]:
import plotly.express as px

fig = px.line(x=arr_k, 
              y=aucs,
              title='Динамика качества при добавлении признаков',
              color_discrete_sequence=["#7E1279"])

fig.update_layout(
    xaxis_title='Количество признаков',          
    yaxis_title='PR-AUC',
    title_font_size=20,     
    font=dict(size=12)     
)

In [ ]:
# график можно посмотреть в Quality_dynamics.png

Явного **ухудшения качества, начиная с какого-то порога, не наблюдается**.\
Просто, начиная с ~30 признаков **качество выходит на ассимптоту**

Давайте подведём итоги по отбору признаков. Назовите преимущества и недостатки каждого из методов. Какой метод привёл к наилучшему качеству?


In [ ]:
display(df_metrics_part3.style
    .background_gradient(subset=['auc_pr_test'], cmap='RdYlGn'))

Что касается качества, все методы, кроме отбора с помощью Random Forest, дали **практически идентичный результат** => есть ядро из 40 сильных признаков.\
Самым долгим с отрывом оказался отбор с помощью метода-обертки.\
Главный минус методов обертки - **вычислительная сложность и скорость выполнения**.\
Метод с отбором с помощью tstat хорош, но **требует нормальности распределения.**\
Метод с корреяциями также дал высокий результат, но он **не учитывает взаимодействия признаков.**\
Метод с весами модели тоже дал высокий результат, но в каких-то других случаях он может **упускать нелинейные закономерности**

Самое интересное, что **наилучшее качество у ЛогРега без отбора признаков**

# Часть 4. Оценка экономического эффекта модели (2 балла)



В данной части мы займемся тем, что от вас скорее всего потребуется на реальной работе (помимо перекладки `json`, разумеется). А именно:
- мы соберем несколько специализированных метрик качества,
- попытаемся настроить модель на максимизацию _прибыли_,
- оценим, сколько вообще получится заработать на этом.

Разумеется, здесь будет сделано множество упрощающих жизнь допущений, но обо всем по порядку. Если вы всё прослушали на экономике, то напомним, что выручка — это сколько денег нам принесли клиенты, а прибыль — выручка за вычетом расходов на зарплату и прочее.


#### __Задание 12. Прогноз по доходам и расходам__ (1 балл)

В этой части мы будем работать с данными [UCI Bank Marketing Dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Этот датасет содержит информацию о банковском телефонном маркетинге.

__Объектом__ здесь является телефонный звонок потенциальному клиенту с предложением некоторой услуги (утверждается, что это краткосрочный депозит). В качестве признакового описания используются характеристики клиента (образование, брак и т.д.), данные о звонке и различные экономические индикаторы - более подробная информация представлена в файле `bank-additional-names.txt`.
__Целевая переменная__ - ответ клиента (согласился ли он открыть депозит?)

In [ ]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
#!unzip bank-additional.zip
df = pd.read_csv('bank-additional-full.csv', sep=';')

Взял описание из интернета

**Демографические признаки:**
- `age` - возраст клиента в годах
- `job` - тип профессии/рода занятий
- `marital` - семейное положение
- `education` - уровень образования
- `default` - наличие кредита в дефолте
- `housing` - наличие жилищного кредита
- `loan` - наличие персонального кредита

**Связанные с текущим контактом:**
- `contact` - тип средства связи
- `month` - последний месяц контакта в году
- `day_of_week` - последний день недели контакта
- `duration` - продолжительность последнего контакта в секундах

**Прочие атрибуты:**
- `campaign` - количество контактов во время этой кампании
- `pdays` - количество дней с последнего контакта
- `previous` - количество контактов до этой кампании
- `poutcome` - результат предыдущей маркетинговой кампании

**Экономические показатели:**
- `emp.var.rate` - коэффициент вариации занятости
- `cons.price.idx` - индекс потребительских цен
- `cons.conf.idx` - индекс потребительского доверия
- `euribor3m` - ставка Euribor за 3 месяца
- `nr.employed` - количество сотрудников

**Целевая переменная:**
- `y` - подписал ли клиент срочный депозит (бинарный)

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['duration', 'y'])
y = (df.y == 'yes')

В этой части не нужно делить выборку - мы будем использовать кросс-валидацию.  Используйте наиболее подходящие с вашей точки зрения параметры и их значения (`shuffle`, `stratify`, число фолдов, ...). По кросс-валидации у вас получится несколько вариантов обучающей и тестовой выборки. Для удобства можно воспользоваться шаблоном ниже, который по ходу выполнения задания будет обрастать функционалом. Как обычно, это необязательно, но сохранять результаты экспериментов очень и очень желательно, в конце мы будем их сравнивать

In [ ]:
# я добавил несколько возможных целевых метрик в этот шаблон

In [ ]:
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score

def cross_validate(
    X,
    y,
    model,
    n_splits=5,
    random_state=None,
    shuffle=False,
    scoring=None # целевая метрика
    # другие аргументы, которые могут вам пригодиться дальше по пунктам
):
    metrics = []
    # или любой другой фолд, посмотрите в model_selection
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

    if scoring is None:
        scoring = ['accuracy', 'precision', 'recall', 'f1', 'average_precision']

    for train_index, test_index in kf.split(X):

        # возьмите датасет и обучите модель
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Обучаем модель
        model.fit(X_train, y_train)


        # для average_precision нужны вероятности
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            # Если модель не имеет predict_proba, используем decision_function
            y_pred_proba = model.decision_function(X_test)
        
        # Предсказываем
        y_pred = model.predict(X_test)
        

        # посчитайте метрики, которые вам нужны и добавьте результаты с каждого фолда
        metric_dict = {
            # "metric_key": metric_value
        }
        if 'accuracy' in scoring:
            metric_dict['accuracy'] = accuracy_score(y_test, y_pred)
        if 'precision' in scoring:
            metric_dict['precision'] = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        if 'recall' in scoring:
            metric_dict['recall'] = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        if 'f1' in scoring:
            metric_dict['f1'] = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        if 'average_precision' in scoring:
            metric_dict['average_precision'] = average_precision_score(y_test, y_pred_proba)    
        
        metrics.append(metric_dict)
    
    return pd.DataFrame(metrics)

Выберите метрику классификации, которая вам кажется подходящей, и обучите логистическую регрессию на каждой обучающей выборке (закодируйте категориальные признаки способом, который выше вам понравился больше всех, отнормируйте числовые, гиперпараметры оставьте по умолчанию), сделайте предсказания для соответствующих тестовых выборок, выведите результаты

На лекции рассказывали, что для подобных задач хорошо подходит **lift** и похожие на него метрики.\
Поэтому я решил использовать **precision_at_k** - доля True Positives среди первых К элементов проранжированного по убыванию вероятности принадлежности к положительному классу ряда объектов.\
Также есть **Average Precision** - метрика, отражающая средний precision на позициях, где есть TP элементы.

In [ ]:
X.head(3)

In [ ]:
target_value = 'admin.'

for col in X.columns:
    if X[col].dtype == 'object':  # проверяем только текстовые столбцы
        if target_value in X[col].values:
            print(f"Значение '{target_value}' найдено в столбце: {col}")
            print(f"Все уникальные значения в '{col}': {X[col].unique()}")

In [ ]:
X = X.replace('admin.', 'administrator')

In [ ]:
df['nr.employed'].unique()

In [ ]:
# подготовка признаков

categorical_features = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']  # категориальные признаки
numeric_features = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'] # числовые признаки

preprocessor12 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)
    ])


preprocessor12.fit(X)
X_transformed = preprocessor12.transform(X)

In [ ]:
model = LogisticRegression(random_state=777)

In [ ]:
X_transformed_df = pd.DataFrame(X_transformed)

In [ ]:
results_ap = cross_validate(
    X_transformed_df, y, model,
    n_splits=5,
    scoring=['average_precision']  # только интересующая нас метрика - average_precision
)

print("Average Precision по фолдам:")
print(results_ap)
print(f"\nСредний Average Precision: {results_ap['average_precision'].mean():.4f}")

Качество ужасное. Посмотрим на баланс классов:

In [ ]:
y.value_counts()

Наблюдается **сильный дисбаланс классов**. Скорее всего, качество низкое из-за него. Попробую обучить лог регрессию с балансировкой классов:

In [ ]:
balanced_lr = LogisticRegression(
    random_state=777,
    class_weight='balanced',  # автоматическая балансировка
    max_iter=1000,
    C=1.0,
    solver='liblinear'
)

In [ ]:
results_ap = cross_validate(
    X_transformed_df, y, balanced_lr,
    n_splits=5,
    scoring=['average_precision']  # только интересующая нас метрика - average_precision
)

print("Average Precision по фолдам:")
print(results_ap)
print(f"\nСредний Average Precision: {results_ap['average_precision'].mean():.4f}")

**Балансировка не помогла**.\
Попробуем прикинуть силу признаков, оценив корреляции

In [ ]:
# Анализ корреляции признаков с целевой переменной

y_numeric = y
correlations = []
for col in X.columns:
    if X[col].dtype in ['int64', 'float64']:
        corr = np.corrcoef(X[col], y_numeric)[0, 1]
        correlations.append((col, corr, abs(corr)))

correlations.sort(key=lambda x: x[2], reverse=True)

print("Топ-10 признаков по корреляции:")
for col, corr, abs_corr in correlations[:10]:
    print(f"{col:20} | Корреляция: {corr:7.3f}")

Признаки **очень слабо коррелированы с таргетом** => такое плохое качество ожидаемо

Допустим, работники вашего колл-центра получают за один звонок клиенту 2 доллара. При согласии клиента на предлагаемые условия он принесет в банк 10 долларов. Предположим, что всем положительным прогнозам ваши сотрудники решили позвонить.

В качестве бизнес-метрики в нашей задаче мы будем считать прибыль aka `profit`, соответственно лучшую модель будем выбирать исходя из этого.
Посчитайте на всех тестовых выборках выручку и сохраните результаты для бизнес-метрики вместе с предыдущей метрикой, которую вы выбрали

Ответьте на вопросы:
- Сколько денег вы в среднем заработаете?
- Какое получилось стандартное отклонение профита?
- Сколько из заработанных денег придётся отдать операторам вашего колл-центра?
- Пропорциональна ли бизнес-метрика выбранной метрике классификации?

In [ ]:
# Функция для расчета прибыли
def calculate_profit(y_true, y_pred_proba, threshold=0.5):

    y_pred = (y_pred_proba >= threshold).astype(int)
    
    n_calls = np.sum(y_pred)
    
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    
    call_cost = -2 * n_calls  
    revenue = 10 * true_positives  
    profit = revenue + call_cost
    
    return profit

In [ ]:
def calculate_costs(y_true, y_pred_proba, threshold=0.5):

    y_pred = (y_pred_proba >= threshold).astype(int)
    
    n_calls = np.sum(y_pred)
    
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    
    call_cost = -2 * n_calls  
    revenue = 10 * true_positives  
    profit = revenue + call_cost
    
    return call_cost

Для подсчета прибыли я изменил cross-validate

In [ ]:
from sklearn.base import clone

In [ ]:
def simple_cross_validate(estimator, X, y, cv=5):
    from sklearn.model_selection import KFold
    from sklearn.metrics import average_precision_score
    
    kf = KFold(n_splits=cv, shuffle=True, random_state=777)
    
    ap_scores = []
    profits = []
    costs = []
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = clone(estimator)
        model.fit(X_train, y_train)
        
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        ap_score = average_precision_score(y_test, y_pred_proba)
        ap_scores.append(ap_score)
        
        profit = calculate_profit(y_test.values, y_pred_proba, threshold=0.5)
        profits.append(profit)

        cost = calculate_costs(y_test.values, y_pred_proba, threshold=0.5)
        costs.append(cost)
    
    return ap_scores, profits, costs

ap_scores, profits, costs = simple_cross_validate(balanced_lr, X_transformed_df, y, cv=5)


In [ ]:
scores_profits_costs = pd.DataFrame()
scores_profits_costs['scores'] = ap_scores
scores_profits_costs['profits'] = profits
scores_profits_costs['costs'] = costs

In [ ]:
print(f'В среднем мы заработаем {scores_profits_costs['profits'].mean()}$')

print(f'Стандартное отклонение прибыли = {round(scores_profits_costs['profits'].std(), 4)}$')

print(f'Отдать операторам придется  в среднем {np.abs(scores_profits_costs['costs'].mean())}$')

In [ ]:
results_ap['profit'] = scores_profits_costs['profits']
results_ap['proportion'] = results_ap['profit'] / results_ap['average_precision']
results_ap

**Нельзя сказать, что метрики пропорциональны**

Внесем некоторую долю случайности. Пусть теперь согласный на условия клиент будет приносить не 10 долларов, а случайную величину, равномерно распределенную в интервале $[0;20)$. Проделайте все те же самые действия. Для имитации реальной ситуации **НЕ** фиксируйте `random_seed` при подсчете выручки с клиента (для разбиения на фолды разумеется оставьте). Что получилось?

In [ ]:
# Функция для расчета прибыли
def calculate_profit_random(y_true, y_pred_proba, threshold=0.5):

    y_pred = (y_pred_proba >= threshold).astype(int)
    
    n_calls = np.sum(y_pred)
    
    true_positives = np.sum((y_pred == 1) & (y_true == 1))


    if true_positives > 0:
        random_revenues = np.random.uniform(0, 20, true_positives)
        total_revenue = np.sum(random_revenues)
    else:
        total_revenue = 0
    
    call_cost = -2 * n_calls   
    profit = total_revenue + call_cost
    
    return profit

In [ ]:
def simple_cross_validate_random_profit(estimator, X, y, cv=5):
    from sklearn.model_selection import KFold
    from sklearn.metrics import average_precision_score
    
    kf = KFold(n_splits=cv, shuffle=True, random_state=777)
    
    ap_scores = []
    profits = []
    costs = []
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = clone(estimator)
        model.fit(X_train, y_train)
        
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        ap_score = average_precision_score(y_test, y_pred_proba)
        ap_scores.append(ap_score)
        
        profit = calculate_profit_random(y_test.values, y_pred_proba, threshold=0.5)
        profits.append(profit)

        cost = calculate_costs(y_test.values, y_pred_proba, threshold=0.5)
        costs.append(cost)
    
    return ap_scores, profits, costs

ap_scores, profits, costs = simple_cross_validate_random_profit(balanced_lr, X_transformed_df, y, cv=5)

In [ ]:
scores_profits_costs = pd.DataFrame()
scores_profits_costs['scores'] = ap_scores
scores_profits_costs['profits'] = profits
scores_profits_costs['costs'] = costs

scores_profits_costs

In [ ]:
results_ap['profit2'] = scores_profits_costs['profits']
results_ap['proportion2'] = results_ap['profit2'] / results_ap['average_precision']
results_ap

In [ ]:
print(f'Средняя прибыль при равномерно распределенном выигрыше = {round(results_ap['profit2'].mean(), 3)}')

**Результаты схожи**, средняя прибыли при случайном выигрыше чуть выше

Настройте по кросс-валидации коэффициент регуляризации модели для максимизации прибыли (считайте как случайную величину выше). Удалось ли получить какой-то выигрыш? При каком коэффициенте регуляризациии прибыль максимальна? Постройте график зависимости ожидаемой прибыли от коэффициента

In [ ]:
def simple_cross_val_profit(estimator, X, y, C_values, cv=5):

    
    kf = KFold(n_splits=cv, shuffle=True, random_state=777)
    
    results = {}
    
    for C in tqdm(C_values):
        profits_fold = []
        
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Создаем модель с конкретным C
            model = clone(estimator)
            model.set_params(C=C)  
            
            model.fit(X_train, y_train)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            
            profit = calculate_profit_random(y_test.values, y_pred_proba, threshold=0.5)
            profits_fold.append(profit)
        
        results[C] = {
            'mean_profit': np.mean(profits_fold),
            'all_profits': profits_fold
        }
    
    return results



In [ ]:
C_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
results = simple_cross_val_profit(balanced_lr, X_transformed_df, y, C_values, cv=5)



In [ ]:
results = pd.DataFrame(results)
results

In [ ]:
maxprofit = results.max(axis=1)['mean_profit']
c_maxprofit = results.idxmax(axis=1)['mean_profit']
print(f'Прибыль максимальна при коэффициенте регуляризации = {c_maxprofit}  и равна {round(results.max(axis=1)['mean_profit'], 3)} $')

**Прибыль выше на ~50$**, выигрыш есть, но он не очень большой

### График зависимости ожидаемой прибыли от коэффициента регуляризации

In [ ]:
import plotly.express as px

In [ ]:
c_reg = np.array(results.columns)
mean_profits = np.array(results.iloc[0].values)



fig = px.line(x=c_reg, 
              y=mean_profits, 
              title='Зависимость ожидаемой прибыли от коэффициента регуляризации',
              color_discrete_sequence=["#098117"])

fig.update_layout(
    xaxis_title='Коэффициент регуляризации',          
    yaxis_title='Ожидаемая прибыль',
    title_font_size=20,     
    font=dict(size=12)     
)

fig.update_traces(
    line_width=2       
)

fig.show()

In [ ]:
# график можно посмотреть в Profit_dynamics_by_reg_coef.png

Для наглядности построю такой же график без учета больших C

In [ ]:
c_reg = np.array(results.columns)
mean_profits = np.array(results.iloc[0].values)



fig = px.line(x=c_reg[:-3], 
              y=mean_profits[:-3], 
              title='Зависимость ожидаемой прибыли от коэффициента регуляризации',
              color_discrete_sequence=["#098117"])

fig.update_layout(
    xaxis_title='Коэффициент регуляризации',          
    yaxis_title='Ожидаемая прибыль',
    title_font_size=20,     
    font=dict(size=12)     
)

fig.update_traces(
    line_width=2       
)

fig.show()

Попробуйте запустить перебор несколько раз. Находится ли каждый раз один и тот же "лучший" коэффициент? Присутствует ли какая-то закономерность? Какие вы можете сделать из этого выводы?

In [ ]:
def simple_cross_val_profit_several_times(estimator, X, y, C_values, cv=5):

    
    kf = KFold(n_splits=cv, shuffle=True, random_state=777)
    
    results = {}
    
    for C in tqdm(C_values):
        profits_fold = []
        
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Создаем модель с конкретным C
            model = clone(estimator)
            model.set_params(C=C)  
            
            model.fit(X_train, y_train)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            
            profit = calculate_profit_random(y_test.values, y_pred_proba, threshold=0.5)
            profits_fold.append(profit)
        
        results[C] = {
            'mean_profit': np.mean(profits_fold),
            'all_profits': profits_fold
        }
        results = pd.DataFrame(results)
    
    return results.idxmax(axis=1)['mean_profit']

In [ ]:
# запускаю функцию 10 раз

[simple_cross_val_profit_several_times(balanced_lr, X_transformed_df, y, C_values, cv=5) for _ in range(10)]

**Вывод**: в большинстве случаев C большой => **сильная регуляризация не нужна**\
Я думаю можно использовать C=10000 или 1000, скорее всего разница будет небольшая

#### __Задание 13. Ключевая метрика__ (1 балл)

Выше мы уже описали примерную экономическую модель вашей задачи. Как вы считаете, что для вашего бизнеса важнее — хороший precision или recall модели? Почему?

__Ответ:__ recall здесь важнее, так как **высокий recall => меньше пропускаем готовых заключить депозит клиентов**\
Издержки на звонок меньше ожидаемой прибыли от заключения депозита, поэтому precision не так важен



> Вспомним, что на самом деле логистическая регрессия предсказывает нам вероятности положительного класса для объекта. Возможно, путем настройки __порога бинаризации__ этих вероятностей мы сможем получить какой-то выигрыш?

Проверьте ваши рассуждения выше с помощью настройки порога бинаризации на кросс-валидации для максимизации прибыли. Воспользуйтесь сеткой от 0 до 1 с шагом 0.01. Напомним, что снижение порога дает нам более высокий recall и более низкий precision, и наоборот. Добавьте новую ML-метрику в ваш CV-пайплайн, найдите такой порог, при котором бизнес-метрика максимальна, и проверьте, связана ли новая ML метрика с профитом

In [ ]:
def simple_cross_val_profit_threshold(estimator, X, y, thresholds, C, cv=5):

    
    kf = KFold(n_splits=cv, shuffle=True, random_state=777)
    
    results = {}
    
    for threshold in tqdm(thresholds):
        profits_fold = []
        recalls_fold = [] 
        
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            # Создаем модель с конкретным C
            model = clone(estimator)
            model.set_params(C=C)  
            
            model.fit(X_train, y_train)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            
            profit = calculate_profit_random(y_test.values, y_pred_proba, threshold=threshold)
            profits_fold.append(profit)

            y_pred = (y_pred_proba >= threshold).astype(int)
            recalls_fold.append(recall_score(y_test, y_pred))
        
        results[threshold] = {
            'mean_profit': np.mean(profits_fold),
            'profis_by_folds': profits_fold,
            'mean_recalls': np.mean(recalls_fold) 
        }
        results = pd.DataFrame(results)
    
    return results


In [ ]:
thresholds = np.arange(0, 1.01, 0.01)
results = simple_cross_val_profit_threshold(balanced_lr, X_transformed_df, y, thresholds, C = 10000, cv=5)

In [ ]:
results

In [ ]:
maxprofit = results.max(axis=1)['mean_profit']
t_maxprofit = results.idxmax(axis=1)['mean_profit']
recall_maxprofit = results[t_maxprofit]['mean_recalls']
print(f'Прибыль максимальна при пороге бинаризации = {t_maxprofit}  и равна {round(maxprofit, 3)} $. Recall в таком случае = {round(recall_maxprofit, 3)}')

**Recall достаточно низкий при максимальной прибыли**. Построим графики зависимости Recall и прибыли от порога бинаризации

In [ ]:
trs = np.array(results.columns)
profits = results.iloc[0].values
recalls = results.iloc[2].values

Постройте график зависимости прибыли от порога бинаризации. Выделите наилучший порог

In [ ]:
fig = px.line(x=trs, 
              y=profits, 
              title='Зависимость ожидаемой прибыли от порога бинаризации',
              color_discrete_sequence=["#7E1279"])

fig.update_layout(
    xaxis_title='Порог бинаризации',          
    yaxis_title='Ожидаемая прибыль',
    title_font_size=20,     
    font=dict(size=12)     
)

fig.add_scatter(
    x=[results.idxmax(axis=1)['mean_profit']], 
    y=[results.max(axis=1)['mean_profit']],
    mode='markers',
    marker=dict(
        color='red',
        size=12,
        symbol='star',  # Звезда вместо круга
        line=dict(color='darkred', width=2)
    ),
    name=f'Максимум: {results.max(axis=1)['mean_profit']:.2f} \
           Порог = {results.idxmax(axis=1)['mean_profit']}'
)

fig.update_layout(
    xaxis_title='Порог бинаризации',          
    yaxis_title='Ожидаемая прибыль',
    title_font_size=20,     
    font=dict(size=12)     
)

fig.update_traces(
    line_width=2       
)

fig.show()

In [ ]:
# график можно посмотреть в Profit_dynamics_by_binarization_threshold.png

In [ ]:
fig = px.line(x=trs, 
              y=recalls, 
              title='Зависимость Recall от порога бинаризации',
              color_discrete_sequence=["#5F1268"])

fig.update_layout(
    xaxis_title='Порог бинаризации',          
    yaxis_title='Recall',
    title_font_size=20,     
    font=dict(size=12)     
)

fig.update_traces(
    line_width=2       
)

fig.show()

In [ ]:
# график можно посмотреть в Recall_dynamics_by_binarization_threshold.png

__Вопрос:__ Замечаете ли вы какую-то закономерность? Для правильного ответа на этот вопрос попробуйте запустить несколько раз и задумайтесь, почему порог получается в какой-то конкретной области?

__Ответ:__ Да, **оптимальный порог всегда в районе 0.6 - 0.7**\
Это говорит о том, что с точки зрения максимизации прибыли, **лучше пропустить несколько клиентов, чем звонить всем подряд**\
Порог 0.6 - 0.7 дает оптимальный **баланс между precision и recall**\
При низком пороге Recall высокий, но слишком много ложных срабатываний => **огромные издержки**

Наконец, чтобы точнее понять, что наша модель лучше исходной, посчитайте среднее и стандартное отклонение по фолдам бизнес-метрики для оптимизированной модели (гиперпараметры + порог) и дефолтной логистической регрессии. Проверьте, действительно ли удалось добиться значимого изменения прибыли — примените какой-либо статистический тест (например, парный t-критерий с $\alpha=0.95$) к метрике, полученной двумя этими моделями

In [ ]:
# обучаю дефолтный логрег

def simple_cross_val_profit_default_df(estimator, X, y, cv=5):
    
    kf = KFold(n_splits=cv, shuffle=True, random_state=777)
    
    profits_fold = []
    recalls_fold = []
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model = clone(estimator)
        model.fit(X_train, y_train)
        
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        profit = calculate_profit_random(y_test.values, y_pred_proba, threshold=0.5)
        profits_fold.append(profit)

        y_pred = (y_pred_proba >= 0.5).astype(int)
        recalls_fold.append(recall_score(y_test, y_pred))
    
    results_def = pd.DataFrame({
        'mean_profit': [np.mean(profits_fold)],
        'profits_by_folds': [profits_fold],
        'mean_recall': [np.mean(recalls_fold)],
    })
    
    return results_def

results_default = simple_cross_val_profit_default_df(balanced_lr, X_transformed_df, y, cv=5)
results_default

In [ ]:
# Оптимизированная лог регрессия
results[t_maxprofit]

In [ ]:
print(f'Ожидаемая прибыль для оптимизированной модели = {np.array(results[t_maxprofit]['profis_by_folds']).mean():.2f}, стандартное отклонение прибыли = {np.array(results[t_maxprofit]['profis_by_folds']).std():.2f}')

print(f'Ожидаемая прибыль для дефолтной модели = {np.array(results_default.iloc[0]['profits_by_folds']).mean():.2f}, стандартное отклонение прибыли = {np.array(results_default.iloc[0]['profits_by_folds']).std():.2f}')

**Средние сильно различаются, посмотрим, статистически значима ли эта разница**

У нас парные наблюдения и маленькая выборка => подходит **ttest_rel**

In [ ]:
from scipy.stats import ttest_rel

profit_model_default = np.array(results_default.iloc[0]['profits_by_folds'])
profit_model_ortimized = np.array(results[t_maxprofit]['profis_by_folds'])

t_stat, p_value = ttest_rel(profit_model_ortimized, profit_model_default, alternative='greater') # проверяю гипотезу что ожидаемая прибыль оптимальной модели выше чем дефолтной

print(f"t-статистика: {t_stat:.3f}")
print(f"p-value: {p_value:.3f}")

In [ ]:
p_value < 0.05

p-value < уровня значимости => нулевая гипотеза отвергается => **Оптимизированная модель значимо лучше**